In [ ]:
import numpy as np
import pandas as pd
from functools import reduce
import time
import sys
import os   
import re
import importlib
import math
import sklearn
from sklearn import preprocessing
from scipy.spatial import Delaunay
from scipy.spatial.transform import Rotation
from scipy import stats
from matplotlib import cm
import matplotlib.animation as animation
# import matplotlib.colors as colors
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

%matplotlib widget

## DO THIS TO RUN R CODE WITHIN PYTHON NOTEBOOK
# %pip install r2py
# os.environ['R_HOME'] = "/usr/local/Cellar/r/4.0.4/lib/R"
# import rpy2.robjects as robjects

## Definitions
### Plot specs

In [ ]:
viewYZ = (0,0)
viewXZ = (0,90)
viewXY = (90,90)
# viewYX = (270,180)
viewYX = (90, 0)
view3Q = (45,45)
view3Qst = (-45,45)
view3Qsw = (135,45)
view3Qpos = (45,45)
view3Qneg = (135,-135)

In [ ]:
colors['red']

### JCS conversion

In [ ]:
def convertDataBetweenJCS(destination_matrix, source_matrix, source_df):
    source_from_dest_matrix = np.matmul(source_matrix, destination_matrix.T)
    rotations = Rotation.from_euler('XYZ', source_df.loc[:,['Rx','Ry','Rz']].to_numpy()*-1, degrees=True)
    converted = rotations*Rotation.from_matrix(source_from_dest_matrix)
    eulers = converted.as_euler('XYZ', degrees=True) *-1   
    result_df = source_df.copy()
    result_df.loc[:,['Rx','Ry','Rz']] = eulers
    return result_df

### Python alpha shapes

In [ ]:
from collections import defaultdict

def alpha_shape_3D(pos, alpha):
    """
    Compute the alpha shape (concave hull) of a set of 3D points.
    Parameters:
        pos - np.array of shape (n,3) points.
        alpha - alpha value.
    return
        outer surface vertex indices, edge indices, and triangle indices
    """

    tetra = Delaunay(pos)
    # Find radius of the circumsphere.
    # By definition, radius of the sphere fitting inside the tetrahedral needs 
    # to be smaller than alpha value
    # http://mathworld.wolfram.com/Circumsphere.html
    tetrapos = np.take(pos,tetra.vertices,axis=0)
    normsq = np.sum(tetrapos**2,axis=2)[:,:,None]
    ones = np.ones((tetrapos.shape[0],tetrapos.shape[1],1))
    a = np.linalg.det(np.concatenate((tetrapos,ones),axis=2))
    Dx = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[1,2]],ones),axis=2))
    Dy = -np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,2]],ones),axis=2))
    Dz = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,1]],ones),axis=2))
    c = np.linalg.det(np.concatenate((normsq,tetrapos),axis=2))
    r = np.sqrt(Dx**2+Dy**2+Dz**2-4*a*c)/(2*np.abs(a))
    # Find tetrahedrals
    tetras = tetra.vertices[r<alpha,:]
    # triangles
    TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
    Triangles = tetras[:,TriComb].reshape(-1,3)
    Triangles = np.sort(Triangles,axis=1)

    # Remove triangles that occurs twice, because they are within shapes
    TrianglesDict = defaultdict(int)
    for tri in Triangles:TrianglesDict[tuple(tri)] += 1
    Triangles=np.array([tri for tri in TrianglesDict if TrianglesDict[tri] ==1])
    #edges
    EdgeComb=np.array([(0, 1), (0, 2), (1, 2)])
    Edges=Triangles[:,EdgeComb].reshape(-1,2)
    Edges=np.sort(Edges,axis=1)
    Edges=np.unique(Edges,axis=0)

    Vertices = np.unique(Edges)
    return Vertices,Edges,Triangles

### Plotting

In [ ]:

def format3dPlot(ax, title, xRange, yRange, zRange, view=None, mode='rotations', joint='glenoid',color='grey', tickInterval=30):
    ax.set_title(title)
    ax.patch.set_alpha(0)
    if view:
        ax.view_init(view[0], view[1])
    if mode == 'rotations':
        if joint=='elbow':
            if view: 
                if view ==(0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- radial deviation   Y  ulnar deviation +', size='small', color=colors['green'])
                    ax.set_zlabel('- flexion   Z  extension +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- flexion   Z   extension +', size='small', color=colors['blue'])  
                elif view == (90,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('+ ulnar deviation   Y   radial deviation -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,0):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- radial deviation   Y   ulnar deviation +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- radial deviation   Y   ulnar deviation +', size='small', color=colors['green'])
                    ax.set_zlabel('- flexion   Z   extension +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- supination   X   pronation +', size='small', color=colors['red'])
                    ax.set_ylabel('+ ulnar deviation   Y   radial deviation -', size='small', color=colors['green'])
                    ax.set_zlabel('+ extension   Z   flexion -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                ax.set_ylabel('+ ulnar deviation   Y   radial deviation -', size='small', color=colors['green'])
                ax.set_zlabel('- flexion   Z   extension +', size='small', color=colors['blue'])
        elif joint=='scap':
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- caudal pitch   Y   cranial pitch +', size='small', color=colors['green'])
                    ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])  
                elif view == (90,0):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('- caudal pitch   Y   cranial pitch +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('+ cranial pitch   Y   caudal pitch -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                    ax.set_ylabel('- caudal pitch   Y   cranial pitch +', size='small', color=colors['green'])
                    ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- lateral roll   X   medial roll +', size='small', color=colors['red'])
                    ax.set_ylabel('+ cranial pitch   Y   caudal pitch -', size='small', color=colors['green'])
                    ax.set_zlabel('+ medial yaw   Z   lateral yaw -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ medial roll   X   lateral roll -', size='small', color=colors['red'])
                ax.set_ylabel('+ cranial pitch   Y   caudal pitch -', size='small', color=colors['green'])
                ax.set_zlabel('- lateral yaw   Z   medial yaw +', size='small', color=colors['blue'])
        else:
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- depression   Y   elevation +', size='small', color=colors['green'])
                    ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue'])
                elif view == (0,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue']) 
                elif view == (90,0):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- depression   Y   elevation +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('+ elevation   Y   depression -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue'])  
                elif view == (45,45):
                    ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                    ax.set_ylabel('- depression   Y   elevation +', size='small', color=colors['green'])
                    ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- supination   X   pronation +', size='small', color=colors['red'])
                    ax.set_ylabel('+ elevation   Y   depression -', size='small', color=colors['green'])
                    ax.set_zlabel('+ protraction   Z   retraction -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ pronation   X   supination -', size='small', color=colors['red'])
                ax.set_ylabel('+ elevation   Y   depression -', size='small', color=colors['green'])
                ax.set_zlabel('- retraction   Z   protraction +', size='small', color=colors['blue'])
        
    elif mode == 'translations':
        if joint=='elbow':
            if view: 
                if view ==(0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- dorsal   Y  ventral +', size='small', color=colors['green'])
                    ax.set_zlabel('- ulnar   Z  radial +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- ulnar   Z   radial +', size='small', color=colors['blue'])  
                elif view == (90,0):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- dorsal   Y   ventral +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('+ ventral   Y   dorsal -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- dorsal   Y   ventral +', size='small', color=colors['green'])
                    ax.set_zlabel('- ulnar   Z   radial +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- distraction   X   compression +', size='small', color=colors['red'])
                    ax.set_ylabel('+ ventral   Y   dorsal -', size='small', color=colors['green'])
                    ax.set_zlabel('+ radial   Z   ulnar -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                ax.set_ylabel('+ ventral   Y   dorsal -', size='small', color=colors['green'])
                ax.set_zlabel('+ radial   Z   ulnar -', size='small', color=colors['blue'])
        elif joint=='scap':
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- lateral   Y   medial +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])  
                elif view == (0,90):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])  
                elif view == (90,0):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('- lateral   Y   medial +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('+ medial   Y   lateral -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (45,45):
                    ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                    ax.set_ylabel('- lateral   Y   medial +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- caudal   X   cranial +', size='small', color=colors['red'])
                    ax.set_ylabel('+ medial   Y   lateral -', size='small', color=colors['green'])
                    ax.set_zlabel('+ dorsal   Z   ventral -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ cranial   X   caudal -', size='small', color=colors['red'])
                ax.set_ylabel('+ medial   Y   lateral -', size='small', color=colors['green'])
                ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
        else:
            if view:
                if view == (0,0):
                    ax.set_xlabel('', size='small', color=colors['red'])
                    ax.set_ylabel('- cranial   Y   caudal +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
                elif view == (0,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue']) 
                elif view == (90,0):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- cranial   Y   caudal +', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue']) 
                elif view == (90,90):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('+ caudal   Y   cranial -', size='small', color=colors['green'])
                    ax.set_zlabel('', size='small', color=colors['blue'])  
                elif view == (45,45):
                    ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                    ax.set_ylabel('- cranial   Y   caudal +', size='small', color=colors['green'])
                    ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])
                elif view == (135,-135):
                    ax.set_xlabel('- distraction   X   compression +', size='small', color=colors['red'])
                    ax.set_ylabel('+ caudal   Y   cranial -', size='small', color=colors['green'])
                    ax.set_zlabel('+ dorsal   Z   ventral -', size='small', color=colors['blue'])
            else:
                ax.set_xlabel('+ compression   X   distraction -', size='small', color=colors['red'])
                ax.set_ylabel('+ caudal   Y   cranial -', size='small', color=colors['green'])
                ax.set_zlabel('- ventral   Z   dorsal +', size='small', color=colors['blue'])

    elif mode == 'pca':
        ax.set_xlabel('PC1', size='small')
        ax.set_ylabel('PC2', size='small')
        ax.set_zlabel('PC3', size='small')
    if view:
        if view == (0,0):
            ax.yaxis.labelpad = 10
            ax.zaxis.labelpad = 10
        elif view == (0,90):
            ax.xaxis.labelpad = 10
        elif view == (90,90):
            ax.xaxis.labelpad = 15
            ax.yaxis.labelpad = 3
        elif view == (90,0):
            ax.xaxis.labelpad = 15
            ax.yaxis.labelpad = 3
    ax.tick_params(reset=True,colors=color, labelsize='x-small')
    ax.set_xlim(xRange[0], xRange[1])
    ax.set_ylim(yRange[0], yRange[1])
    ax.set_zlim(zRange[0], zRange[1])  
    xTicks = np.arange(xRange[0],xRange[1]+1, tickInterval) if view and view!=(0,0) else []
    yTicks = np.arange(yRange[0],yRange[1]+1, tickInterval) if view and view!=(0,90) else []
    zTicks = np.arange(zRange[0],zRange[1]+1, tickInterval) if view and view!=(90,90) and view!=(90,0) else []
    xRotation = 45 if view and (view == (0,90) or view == (90,90)) else 0
    yRotation = 45 if view and view == (0,0) else 0
    ax.set_xticks(xTicks)
    ax.set_yticks(yTicks)
    ax.set_zticks(zTicks)
#     ax.set_xticklabels(rotation = xRotation, va ='top', ha='center')
#     ax.set_yticklabels(rotation = yRotation, va ='top', ha='center')
    ax.w_xaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_yaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_zaxis.set_pane_color((1.0, 1.0, 1.0, 0.0)) 
    rangeX, rangeY, rangeZ = abs(xRange[0]-xRange[1]), abs(yRange[0]-yRange[1]), abs(zRange[0]-zRange[1])
    minRange = min(rangeX, rangeY, rangeZ)
    xAspect, yAspect, zAspect = rangeX/minRange, rangeY/minRange, rangeZ/minRange
    ax.set_box_aspect([xAspect,yAspect,zAspect])
    ax.w_xaxis.line.set_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_yaxis.line.set_color((1.0, 1.0, 1.0, 0.0)) 
    ax.w_zaxis.line.set_color((1.0, 1.0, 1.0, 0.0))

    ax.minorticks_off()
    return ax

def on_move(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
    else:
        return
    fig.canvas.draw_idle()
    

def on_move4(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
        ax2.view_init(elev=ax0.elev, azim=ax0.azim)
        ax3.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
        ax2.view_init(elev=ax1.elev, azim=ax1.azim)
        ax3.view_init(elev=ax1.elev, azim=ax1.azim)
    elif event.inaxes == ax2:
        ax0.view_init(elev=ax2.elev, azim=ax2.azim)
        ax1.view_init(elev=ax2.elev, azim=ax2.azim)
        ax3.view_init(elev=ax2.elev, azim=ax2.azim)
    elif event.inaxes == ax3:
        ax0.view_init(elev=ax3.elev, azim=ax3.azim)
        ax1.view_init(elev=ax3.elev, azim=ax3.azim)
        ax2.view_init(elev=ax3.elev, azim=ax3.azim)
    else:
        return
    fig.canvas.draw_idle()
    
def on_move6(event):
    if event.inaxes == ax0:
        ax1.view_init(elev=ax0.elev, azim=ax0.azim)
        ax2.view_init(elev=ax0.elev, azim=ax0.azim)
        ax3.view_init(elev=ax0.elev, azim=ax0.azim)
        ax4.view_init(elev=ax0.elev, azim=ax0.azim)
        ax5.view_init(elev=ax0.elev, azim=ax0.azim)
    elif event.inaxes == ax1:
        ax0.view_init(elev=ax1.elev, azim=ax1.azim)
        ax2.view_init(elev=ax1.elev, azim=ax1.azim)
        ax3.view_init(elev=ax1.elev, azim=ax1.azim)
        ax4.view_init(elev=ax1.elev, azim=ax1.azim)
        ax5.view_init(elev=ax1.elev, azim=ax1.azim)
    elif event.inaxes == ax2:
        ax0.view_init(elev=ax2.elev, azim=ax2.azim)
        ax1.view_init(elev=ax2.elev, azim=ax2.azim)
        ax3.view_init(elev=ax2.elev, azim=ax2.azim)
        ax4.view_init(elev=ax2.elev, azim=ax2.azim)
        ax5.view_init(elev=ax2.elev, azim=ax2.azim)
    elif event.inaxes == ax3:
        ax0.view_init(elev=ax3.elev, azim=ax3.azim)
        ax1.view_init(elev=ax3.elev, azim=ax3.azim)
        ax2.view_init(elev=ax3.elev, azim=ax3.azim)
        ax4.view_init(elev=ax3.elev, azim=ax3.azim)
        ax5.view_init(elev=ax3.elev, azim=ax3.azim)
    elif event.inaxes == ax4:
        ax0.view_init(elev=ax4.elev, azim=ax4.azim)
        ax1.view_init(elev=ax4.elev, azim=ax4.azim)
        ax2.view_init(elev=ax4.elev, azim=ax4.azim)
        ax3.view_init(elev=ax4.elev, azim=ax4.azim)
        ax5.view_init(elev=ax4.elev, azim=ax4.azim)
    elif event.inaxes == ax5:
        ax0.view_init(elev=ax5.elev, azim=ax5.azim)
        ax1.view_init(elev=ax5.elev, azim=ax5.azim)
        ax2.view_init(elev=ax5.elev, azim=ax5.azim)
        ax3.view_init(elev=ax5.elev, azim=ax5.azim)
        ax4.view_init(elev=ax5.elev, azim=ax5.azim)
    else:
        return
    fig.canvas.draw_idle()
    
def addCosGrid(gridAx, xRange, yRange, zRange, interval, xLevels=1, alpha=0, **kwargs):  
    xMin = math.floor(xRange[0])
    xMax = math.ceil(xRange[1])+1
    yMin = math.floor(yRange[0])
    yMax = math.ceil(yRange[1])+1
    zMin = math.floor(zRange[0])
    zMax = math.ceil(zRange[1])+1
    zs= np.arange(zMin, zMax+1, 1)
    ys= np.arange(yMin, yMax, 1)
    xSize = abs(xMin)+abs(xMax)
    ySize = abs(yMin)+abs(yMax)
    zSize = abs(zMin)+abs(zMax)
    alphas = np.ones(zSize)*alpha
    zz, yy = np.meshgrid(zs, ys)
    czz = zz*np.cos(np.radians(yy))
    xMaxMinMax = max((abs(xMin),abs(xMax)))
    if xLevels <2:
        xs = np.zeros(xSize)
    else:
        xs = np.linspace(xMaxMinMax*-1, xMaxMinMax, xLevels)
    for xLevel in list(range(xLevels)):
        xx = np.ones((czz.shape[0],czz.shape[1]))*xs[xLevel]
        gridAx.plot_wireframe(xx, yy, czz, rcount = zSize/interval, ccount=ySize/interval, **kwargs) 



## Imports
### Import model data

In [ ]:
# phil_matrix = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]
# rob_tegu_matrix = np.array([[0.917443254918, -0.0507813546731, -0.39461263034, 0.0], [-0.0504523728816, 0.968965969705, -0.241990718053, 0.0], [0.394654826496, 0.2419218956, 0.886409253309, 0.0], [2.88657986403e-15, 3.5527136788e-15, 8.881784197e-16, 1.0]])[0:3,0:3]
# rob_opossum_matrix = np.array([[0.988997647945, 0.0952296421902, 0.113203213768, 0.0], [-0.144901479974, 0.777702952339, 0.611703914508, 0.0], [-0.0297861286566, -0.6213770459, 0.782945306754, 0.0], [0.0, -2.22044604925e-16, -8.881784197e-16, 1.0]])[0:3,0:3]
# rob_opossum_matrix1RLflip = np.array([[-0.988997647945, -0.0952296421902, -0.113203213768, 0.0], [0.144901479974, -0.777702952339, -0.611703914508, 0.0], [-0.0297861286566, -0.6213770459, 0.782945306754, 0.0], [0.0, -2.22044604925e-16, -8.881784197e-16, 1.0]])[0:3,0:3]


# df_M_S = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/Tegu_ROM_output_210712_correct.csv',index_col=None)
# df_M_D = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/Opossum_ROM_output_210713_correct.csv',index_col=None)
# df_M_S.loc[:,'species'] = 'Tegu'
# df_M_D.loc[:,'species'] = 'Opossum'
# df_M = pd.concat([df_M_S,df_M_D])
# df_M.columns = ['rx', 'ry', 'rz', 'species']
# df_M['radRy'] = np.radians(df_M['ry'])
# df_M['cosRy'] = np.cos(df_M['radRy'])
# df_M['RzcosRy'] = df_M['cosRy'].multiply(df_M['rz'])
# df_M['crz'] = df_M['RzcosRy']
# df_M_S = df_M.loc[df_M['species']=='Tegu']
# df_M_D = df_M.loc[df_M['species']=='Opossum']

In [811]:
rob_aliases = { 'pec_crn':r'([a-z]*peca[a-z]*)',
            'pec_int':r'([a-z]*pecm[a-z]*)',
            'pec_cdl':r'([a-z]*pecp[a-z]*)',
            'lat_crn':r'([a-z]*lada[a-z]*)',
            'lat_int':r'([a-z]*ladm[a-z]*)',
            'lat_cdl':r'([a-z]*ladp[a-z]*)',
            'dac':r'([a-z]*dac[a-z]*)',
            'dsc':r'([a-z]*dsc[a-z]*)',
            'dcl':r'([a-z]*dcl[a-z]*)',
            'bic':r'([a-z]*biculn[a-z]*)',
            'bicr':r'([a-z]*bicrad[a-z]*)',
            'bra':r'([a-z]*bra[a-z]*)',
            'cbb':r'([a-z]*cbb[a-z]*)',
            'trs':r'([a-z]*trs[a-z]*)', 
            'trl':r'([a-z]*trl[a-z]*)', 
            'trm':r'([a-z]*trm[a-z]*)', 
            'tmj':r'([a-z]*tmj[a-z]*)',
            'tmn':r'([a-z]*tmn[a-z]*)',
            'isp':r'([a-z]*isp[a-z]*)',
            'ssp':r'([a-z]*ssp[a-z]*)',
            'sbs':r'([a-z]*sbs[a-z]*)'}

def findComparisons(phil_df, animal, side):
    filt_df = phil_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)]
    phil_mmas = [col for col in filt_df.columns if 'Xma' in col and 'scaled' not in col]
    phil_mmas = [col.rsplit('_',1)[0] for col in phil_mmas if pd.notnull(filt_df[col]).all()]
    print(phil_mmas)
    return phil_mmas

def makeMMA3plot(save_dir, phil_df, rob_df, animal, side, phil_muscle, rob_muscle=None, fig_title=None, plot_suffix=''):
    phil_df = phil_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)].reset_index()
    rob_df = rob_df.loc[phil_df.index]
    fig, ax = plt.subplots(3,1,figsize=(16,8), sharey='all', sharex='all')
    rob_muscle = rob_muscle if rob_muscle else phil_muscle
    plt.suptitle(fig_title if fig_title else phil_muscle)
    ax[0].plot(phil_df[phil_muscle+'_Xma'], linestyle='solid', color='red')
    ax[0].plot(rob_df['model_'+rob_muscle+'_glen_Xma'], linestyle='dotted', color='red')
    ax[1].plot(phil_df[phil_muscle+'_Yma'], linestyle='solid', color='green')
    ax[1].plot(rob_df['model_'+rob_muscle+'_glen_Yma'], linestyle='dotted', color='green')
    ax[2].plot(phil_df[phil_muscle+'_Zma'], linestyle='solid', color='blue')
    ax[2].plot(rob_df['model_'+rob_muscle+'_glen_Zma'], linestyle='dotted', color='blue')
    plt.tight_layout()
    plt.savefig(os.path.join(save_dir,phil_muscle+plot_suffix+'.png'), dpi=150, transparent=True)
    return ax

def makeAllComparisonsForAnimalSide(phil_df, rob_df, animal, side, save_dir):
    plt.close('all')
    muscle_list = findComparisons(phil_df, animal, side)
    for muscle in muscle_list:
        makeMMA3plot(save_dir, phil_df, rob_df, animal, side, muscle)

def getPercentageDifference(phil_df, rob_df, animal, side, phil_muscle, rob_muscle):
    rob_df = rob_df.loc[(phil_df['animal']==animal)&(phil_df['side']==side)]
    difference = phil_df[phil_muscle] - rob_df[rob_muscle]
    percentage_difference = difference/phil_df[phil_muscle]*100
    return filter_outliers(percentage_difference)


def getRobAliases(aliasDict, rob_df):
    muscleNamesList = set(rob_df.columns)
    result = {}
    for (key,value) in aliasDict.items():
        matches = [re.search(value, name, re.IGNORECASE) for name in muscleNamesList]
        matched_names = [name.group(0) for name in matches if name]
        result[key] = matched_names
    all_matches = {item for sublist in result.values() for item in sublist}
    return result

def dictFromScanDir(directory, extension):
    key_dict ={'X':'X','Y':'Y','Z':'Z','h':'length'}
    file_dict={'Acromion':{},'Glenoid':{},'Elbow':{},'Length':''}
    for root, dirs, files in os.walk(directory):
        for name in files:
            if name.lower().endswith(extension):
                filename = os.path.join(root, name)
                joint = 'Acromion' if ('cromion' in name) else 'Glenoid' if ('leno' in name) else 'Elbow' if ('lbow' in name) else 'Length' if ('ength' in name) else None
                if joint in ['Acromion','Glenoid','Elbow']:
                    axis_key = key_dict[os.path.splitext(name)[0][-1]]
                    file_dict[joint][axis_key]=filename
                elif joint == 'Length':
                    file_dict['Length'] =filename
            else:
                continue
    return(file_dict)

def dfs_from_sto_dir(sto_dir, side_name):
    sto_dict = dictFromScanDir(sto_dir, '.sto')
    dfs = []
    dfs.append(import_sto(sto_dict['Length'], 'Length', side_name))
    for joint in ['Acromion','Glenoid','Elbow']:
        for axis in ['X','Y','Z']:
            dfs.append(import_sto(sto_dict[joint][axis], joint, side_name, axis=axis))
    big_df = pd.concat(dfs, axis=1)
    return big_df
    
def dropMMA0s(df):
    df = df.replace(0.0, np.nan)
    df = df.dropna(how='any', axis=0)
    df = df.replace(np.nan, 0.0)
    return df

def import_sto(path, joint, side_name, axis=None):
    axis_dict = {'X':'_Xma', 'Y':'_Yma', 'Z':'_Zma'}
    df = pd.read_csv(path, sep='\t', index_col=None, skiprows=np.linspace(0,10))
    actual_side = 'right' if side_name=="left" else 'left'
    df = df.filter(like=actual_side)
    axis_name = axis_dict[axis] if axis else ''
    rename_map = reverseAliasMap(getRobAliases(rob_aliases, df), add_split=False)
    df = df.rename(columns=rename_map)
    joint_name = '_elbo' if joint == 'Elbow' else '_acro' if joint == 'Acromion' else '_glen' if joint == 'Glenoid' else '_length' if joint == 'Length' else ''
    df.columns = ['model_'+col+joint_name+axis_name for col in df.columns]
    df = df.loc[:, (df != 0).any(axis=0)]
    return df

rob85R = dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP85L', 'right')
rob101R= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP101L', 'right')
rob92R= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP92L', 'right')
rob88R= dfs_from_sto_dir('/Users/phil/Desktop/rotoscoping/robData/SEP88L', 'right')

In [832]:
dfD.loc[(dfD['animal']=='88')&(dfD['side']=='R')]['pec_cdl_Xma']


2225   NaN
2226   NaN
2227   NaN
2228   NaN
2229   NaN
        ..
2928   NaN
2929   NaN
2930   NaN
2931   NaN
2932   NaN
Name: pec_cdl_Xma, Length: 708, dtype: float64

#### 88R comparisons

In [794]:
meta_88R = {
                'animal':'88',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/88R'
           }
onBone88R = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/88R/88RonBone.csv', 
                     index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_88R['animal'],side=meta_88R['side']))
comparisons_88R = findComparisons(dfD, meta_88R['animal'], meta_88R['side'])

##trs, dac and dsc redone on bone

makeMMA3plot(meta_88R['folder'], dfD, rob88R, meta_88R['animal'], meta_88R['side'], 'pec_crn', 'pec_int', fig_title='Pectoralis (intermediate)')
makeMMA3plot(meta_88R['folder'], onBone88R, rob88R, meta_88R['animal'], meta_88R['side'], 'trs', fig_title='Triceps (scapular, bone origin)')
makeMMA3plot(meta_88R['folder'], dfD, rob88R, meta_88R['animal'], meta_88R['side'], 'trs', fig_title='Triceps (scapular, fleshy origin)',plot_suffix='_fsh')

['pec_crn', 'trs', 'dsc', 'dac']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

#### 85R comparisons

In [833]:
new85Rlat = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newlat85R.csv', 
                     index_col=0)
new85Rtrs = pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R/newtrs85R.csv', 
                     index_col=0)
meta_85R = {
                'animal':'85',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/85R'
           }
onBone85R = dropMMA0s(pd.concat([new85Rlat,new85Rtrs],axis=1).rename(columns=reverse_alias_map).assign(animal=meta_85R['animal'],side=meta_85R['side']))

comparisons_85R = findComparisons(dfD, meta_85R['animal'], meta_85R['side'])

makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'pec_crn',  fig_title='Pectoralis (cranial)')
makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'pec_int',  fig_title='Pectoralis (intermediate)')
makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'pec_cdl',  fig_title='Pectoralis (caudal)')

makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_crn',  fig_title='Latissimus (cranial, bone insertion)')
makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_cdl',  fig_title='Latissimus (caudal, bone insertion)')

makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_crn',  fig_title='Latissimus (cranial, fleshy insertion)',plot_suffix='_fsh')
makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'lat_cdl',  fig_title='Latissimus (caudal, fleshy insertion)',plot_suffix='_fsh')

makeMMA3plot(meta_85R['folder'], dfD, rob85R, meta_85R['animal'], meta_85R['side'], 'trs',  fig_title='Triceps (scapular, fleshy origin)',plot_suffix='_fsh')
makeMMA3plot(meta_85R['folder'], onBone85R, rob85R, meta_85R['animal'], meta_85R['side'], 'trs',  fig_title='Triceps (scapular, bone origin)')

['lat_crn', 'lat_cdl', 'pec_crn', 'pec_int', 'pec_cdl', 'trs', 'dsc', 'dac']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

#### 101R comparisons

In [827]:
meta_101R = {
                'animal':'101',
                'side':'R', 
                'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/101R'
           }

onBone101Rdb = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/101R/101Ronbone_dscbic.csv', 
                     index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_101R['animal'],side=meta_101R['side']))
onBone101R = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/101R/101Ronbone.csv', 
                     index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_101R['animal'],side=meta_101R['side']))
comparisons_101R = findComparisons(dfD, meta_101R['animal'], meta_101R['side'])


makeMMA3plot(meta_101R['folder'], dfD, rob101R, meta_101R['animal'], meta_101R['side'], 'pec_crn', fig_title='Pectoralis (cranial)')
makeMMA3plot(meta_101R['folder'], dfD, rob101R, meta_101R['animal'], meta_101R['side'], 'pec_cdl', fig_title='Pectoralis (caudal)')
makeMMA3plot(meta_101R['folder'], onBone101R, rob101R, meta_101R['animal'], meta_101R['side'], 'trs', fig_title='Triceps (scapular, bone origin)', plot_suffix='')
makeMMA3plot(meta_101R['folder'], dfD, rob101R, meta_101R['animal'], meta_101R['side'], 'trs', fig_title='Triceps (scapular, fleshy origin)', plot_suffix='_fsh')
makeMMA3plot(meta_101R['folder'], onBone101R, rob101R, meta_101R['animal'], meta_101R['side'], 'dsc', fig_title='Deltoid (scapular)')
makeMMA3plot(meta_101R['folder'], dfD, rob101R, meta_101R['animal'], meta_101R['side'], 'dac', fig_title='Deltoid (acromial)')
makeMMA3plot(meta_101R['folder'], dfD, rob101R, meta_101R['animal'], meta_101R['side'], 'bic', fig_title='Biceps, fleshy origin', plot_suffix='_fsh')
makeMMA3plot(meta_101R['folder'], onBone101Rdb, rob101R, meta_101R['animal'], meta_101R['side'], 'bic', fig_title='Biceps, bone origin')
makeMMA3plot(meta_101R['folder'], onBone101Rdb, rob101R, meta_101R['animal'], meta_101R['side'], 'bic', fig_title='Biceps, bone origin and insertion',plot_suffix='_bb')



['pec_crn', 'pec_int', 'pec_cdl', 'trs', 'dsc', 'dac', 'bic']


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

...

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

array([<AxesSubplot:>, <AxesSubplot:>, <AxesSubplot:>], dtype=object)

#### 92R comparisons

In [ ]:
# meta_92R = {
#                 'animal':'92',
#                 'side':'R', 
#                 'folder':'/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/92R'
#            }
# # onBone92R = dropMMA0s(pd.read_csv('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/MMAcomparisons/92R/92RonBone.csv', 
# #                      index_col=0).rename(columns=reverse_alias_map).assign(animal=meta_92R['animal'],side=meta_92R['side']))
# comparisons_92R = findComparisons(dfD, meta_92R['animal'], meta_92R['side'])

# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'pec_crn', fig_title='Pectoralis (cranial)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'pec_int', fig_title='Pectoralis (intermediate)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'pec_cdl', fig_title='Pectoralis (caudal)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'dsc', fig_title='Deltoid (scapular)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'dac', fig_title='Deltoid (acromial)')
# makeMMA3plot(meta_92R['folder'], dfD, rob92R, meta_92R['animal'], meta_92R['side'], 'bic', fig_title='Biceps (fleshy origin)', plot_suffix='_fsh')
# # makeMMA3plot(meta_92R['folder'], onBone92R, rob92R, meta_92R['animal'], meta_92R['side'], 'bic', fig_title='Biceps (bone origin)')



- Rob from Phil is always the reciprocal/inverse of Phil from Rob
- Offset between proximal JCS is always the same
- Can get offset from proximal JCS of any frame as long as we use the correct multiplication order
- Correct order for our purposes (A rotation to match target B rotation) is Mat_B * Mat_A.inverse()
- This is different from the relative rotation (B rotation from A rotation), which is Mat_A.inverse() * Mat_B
- get Rob from Phil JCS rotations by getting zyx order Eulers from MTransformationMatrix of robFromPhil
- get Phil from Rob JCS rotations by getting xyz order Eulers from MTransformationMatrix of philFromRob
    - xyz rob from phil = [4.700399999999999, 11.080600000000002, -30.509199999999996] zyx rob from phil = [9.737539736905026, 7.089631542779325, -31.570056869286752]
    - zyx phil from rob = [-9.737539736905022, -7.089631542779322, 31.570056869286752] xyz phil from rob = [-4.700399999999999, -11.0806, 30.509199999999996]
- all good so far, am able to use these matrices to snap rob jcs to phil jcs and vice versa. But unable to match JCS locator data. 
- eventually able to recreate JCS locator data by going step by step following JAxesDual source code, end up with same values as negative of robDistFromRobProx25_rot.as_euler('XYZ',degrees=True). ONLY WORKS FOR KEYED FRAMES! for interpolated frames, channel editor gives different value from script
- def checkFrame():
    oldMatrix = xform('robJCS', ws=1, q=1, m=1)
    oldMatrix1 = getAttr('robJCS.worldMatrix')
    axesDurChild = dt.Matrix(xform('robJCS', ws=1, q=1, m=1)).transpose()
    axesDurParent = dt.Matrix(xform('robJCSProx', ws=1, q=1, m=1)).transpose()
    outChild = axesDurParent.transpose()*axesDurChild
    newMatrix = outChild.transpose()
    test = dt.TransformationMatrix(newMatrix)
    testDeg = [math.degrees(rad) for rad in test.euler]
    return testDeg
- need to use 'XYZ' to specify intrinsic rotations when importing eulers from Maya output, and need to flip sign of rotations or else transpose the resulting matrices.
- errors before: didn't know how to bring in eulers to scipy, possibly frame 1 keyed JCS, also interpolated frames



### Import shoulder and elbow JCS data
NB: l and r are swapped in XROMM data. variable names reflect swapped aka wrong sides. JCS are set up such that rotations are correct but left-side (labeled right-side) translations must be reversed. Additionally, these rotations were output using a - extend + flex elbow JCS, must be reversed in code


In [692]:
jcs85R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_right_triceps_lat-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs88L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_left_teresmaj_pec-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs88R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_trilong_pec-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs92R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv92_biceps_3pec_delt-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs101L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101_11Apr-Phil-2020-04-13-diff/maya/JCS-withScap.csv', 
                     index_col=0)
jcs105R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm105-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs126 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm126-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs126L = jcs126.filter(like='Ldata', axis=1)
jcs126R = jcs126.filter(like='Rdata', axis=1)   
jcs85L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_left_biceps_teres_lat-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs101R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101right_biceps_triceps-Phil-2020-06-08/maya/JCS-withScap.csv', 
                     index_col=0)
jcs108 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm108-Phil-2020-06-09/maya/JCS-withScap.csv', 
                     index_col=0)
jcs108L = jcs108.filter(like='ldata', axis=1)
jcs108R = jcs108.filter(like='rdata', axis=1)


jcs_dfs = {'jcs_85_R':jcs85R, 'jcs_85_L':jcs85L, 'jcs_88_L':jcs88L, 'jcs_88_R':jcs88R, 'jcs_92_R':jcs92R, 'jcs_101_L':jcs101L, 'jcs_101_R':jcs101R, 'jcs_105_R':jcs105R, 'jcs_108_L':jcs108L, 'jcs_108_R':jcs108R, 'jcs_126_L':jcs126L, 'jcs_126_R':jcs126R}
jcs_dfs_cols = {item for sublist in [list(df.columns) for df in jcs_dfs.values()] for item in sublist}
jcs_dfs_cols
# jcs_cols = ['Tx','Ty','Tz','Rx','Ry','Rz']
species_dict = {'85':'Opossum','88':'Opossum','92':'Opossum','101':'Opossum','105':'Tegu','108':'Tegu','126':'Tegu'}

#measurements are correct, no need to flip
humeral_lengths = {'85_R':56.750, '85_L':56.620, '88_R':54.015, '88_L':53.454, '92_R':55.358, '101_R':59.293,'101_L':60.039,'105_R':46.588,'108_R':42.134,'108_L':42.824,'126_R':43.666,'126_L':43.296}
humeral_scalefactor = {'85_R':66.639, '85_L':66.639, '88_R':66.639, '88_L':66.639, '92_R':66.639, '101_R':66.639,'101_L':66.639,'105_R':43.5175,'108_R':43.5175,'108_L':43.5175,'126_R':43.5175,'126_L':43.5175}
humeral_volumes = {'85_R':1963.100, '85_L':1734.652, '88_R':1947.153, '88_L':1485.409, '92_R':1408.360, '101_R':2228.326,'101_L':2072.598,'105_R':947.092,'108_R':1345.199,'108_L':1325.958,'126_R':1064.988,'126_L':1061.491}
humeral_volumes_cuberoot = {key:humeral_volumes[key]**(1/3) for key in humeral_volumes}

colors = {'red':'#B74B4B', 'green':'#8EC15A', 'blue':'#5083D2'}
orangeYellowSeafoam = ['#54100F','#9C4622','#DC8528','#FFD793','#BCE1D0','#3CA9A6','#08676A','#012C2D']
redYellowBlue = ['#451021','#A62547','#E47961','#F8D4AC','#BDE2CF','#559FBA','#1C5796','#012C2D']
teguColor = orangeYellowSeafoam[5]
opossumColor = orangeYellowSeafoam[3]
plt.rcParams.update({
    'grid.linewidth':  0,
    'grid.color':  'lightgrey',
    'savefig.facecolor': (0.0, 0.0, 0.0, 0.0), 
    'savefig.transparent': True,
})




def hexToRgb(hexStr):
    decR = int(hexStr[1:3],16)
    decG = int(hexStr[3:5],16)
    decB = int(hexStr[5:7],16)
    decTup = (decR, decG, decB)
    return decTup


def generateNewCols(inputDict, mode='jcs'):
    newDict = inputDict.copy() 
    for (key, value) in newDict.items():
#         if mode=='mma':
#             newDict[key] *= 10
        newDict[key].loc[:,['frame']] = newDict[key].index.values.reshape(-1,1)
        newDict[key].loc[:,['animal']] = key.split('_')[1]
        newDict[key].loc[:,['side']] = key.split('_')[-1]
        newDict[key].loc[:,['species']] = species_dict[key.split('_')[1]]
        newDict[key].loc[:,['humeralLength']] = humeral_lengths[key.split('_',1)[1]]
        newDict[key].loc[:,['humeralLengthFactor']] = humeral_scalefactor[key.split('_',1)[1]]
        newDict[key].loc[:,['humeralVolCubeRoot']] = humeral_volumes_cuberoot[key.split('_',1)[1]]
        newDict[key].loc[:,['UID']] = newDict[key]['species']+newDict[key]['animal']+'_frame'+newDict[key]['frame'].astype(str)
        newDict[key] = newDict[key].set_index('UID')
    return newDict

def renameAndDrop(inputDict):
    newDict = inputDict.copy() 
    for (key, value) in newDict.items():
        newDict[key] = newDict[key].rename(columns = lambda x: re.sub('(stern)+.*(scap)+.*data','acromion',x))
        newDict[key] = newDict[key].rename(columns = lambda x: re.sub('(scap)+.*(hum)+.*data','glenoid',x))
        newDict[key] = newDict[key].rename(columns = lambda x: re.sub('(hum)+.*(uln)+.*data','elbow',x))
        newDict[key] = newDict[key].replace(0.0, np.nan)
        newDict[key] = newDict[key].dropna(how='any', axis=0)
        newDict[key] = newDict[key].replace(np.nan, 0.0)
    return newDict


### Import MMA data

In [693]:
ma85R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_right_triceps_lat-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma88L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_left_teresmaj_pec-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma88R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv88_trilong_pec-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma92R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv92_biceps_3pec_delt-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma101L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101_11Apr-Phil-2020-04-13-diff/maya/momentArms.csv', 
                     index_col=0)
ma105R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm105-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma126 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm126-Phil-2020-06-09/maya/momentArms.csv', 
                     index_col=0)
ma126L = ma126.filter(like='L_jnt', axis=1)
ma126R = ma126.filter(like='R_jnt', axis=1)
ma85L = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/dv85_left_biceps_teres_lat-Phil-2020-06-09/maya/MomentArms.csv', 
                     index_col=0)
ma101R = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/possum101right_biceps_triceps-Phil-2020-06-08/maya/MA.csv', 
                     index_col=0)
ma108 = pd.read_csv('/Users/phil/Google Drive/Development/DeepLabCut/dev/sm108-Phil-2020-06-09/maya/MomentArms.csv', 
                     index_col=0)
ma108L = ma108.filter(like='l_jnt', axis=1)
ma108R = ma108.filter(like='r_jnt', axis=1)

ma_dfs = {'ma_85_R':ma85R, 'ma_85_L':ma85L, 'ma_88_L':ma88L, 'ma_88_R':ma88R, 'ma_92_R':ma92R, 'ma_101_L':ma101L, 'ma_101_R':ma101R, 'ma_105_R':ma105R, 'ma_108_L':ma108L, 'ma_108_R':ma108R, 'ma_126_L':ma126L, 'ma_126_R':ma126R}

def filterMaByJcs(maDict, jcsDict):
    newDict = maDict.copy() 
    for (key, value) in maDict.items():
        newDict[key] = newDict[key][newDict[key].index.isin(jcsDict['jcs'+'_'+key.split('_',1)[-1]].index)]
    return newDict

ma_dfs = filterMaByJcs(ma_dfs, jcs_dfs)

### Process raw data

In [694]:
aliases = { 'pec_crn':r'(?=.*pec)(?=.*cr).*',
            'pec_int':r'(?=.*pec)(?=.*int).*',
            'pec_cdl':r'(?=.*pec)(?=.*cau|.*c.*d.*l).*',
            'lat_crn':r'(?=.*lat)(?!.*cau|.*c.*d.*l).*',
            'lat_cdl':r'(?=.*lat)(?=.*cau|.*c.*d.*l).*',
            'dac':r'(.*dac|.*acr).*',
            'dsc':r'(.*dsc|.*sc.*p).*',
            'bic':r'(.*bic).*',
            'trs':r'(.*tri).*', 
            'tmj':r'(.*tmj|.*teres).*' }

def getMuscleNames(maDict):
    muscle_names = [set(value.columns) for (key,value) in maDict.items()]
    muscle_names = set().union(*muscle_names)
    muscle_names = {name for name in muscle_names}
    return muscle_names

def getAliases(aliasDict, muscleNamesList):
    result = {}
    for (key,value) in aliasDict.items():
        matches = [re.match(value, name, re.IGNORECASE) for name in muscleNamesList]
        matched_names = [name.group(0) for name in matches if name]
        result[key] = matched_names
    all_matches = {item for sublist in result.values() for item in sublist}
    missedA = all_matches.difference(muscleNamesList)
    missedB = muscleNamesList.difference(all_matches)
    missed = missedA.union(missedB)
    if missed:
        print('Failed to match some muscle names; edit regexes and try again: ' +str(missed))
    return result

def reverseAliasMap(mappedAliasesDict, add_split=True):
    result = {}
    for (key, value) in mappedAliasesDict.items():
        for subvalue in value:
            result[subvalue]=key+'_'+subvalue.split('.')[-1] if add_split else key
    return result

def renameMusclesWithMap(maDict, nameMap):
    newDict = maDict.copy() 
    for (key, value) in newDict.items():
        newDict[key] = newDict[key].rename(columns=nameMap)
    return newDict

jcs_dfs = generateNewCols(jcs_dfs, mode='jcs')
jcs_dfs = renameAndDrop(jcs_dfs)
jcs_concat = pd.concat(jcs_dfs.values())


#flip Y and Z for elbow to match latest CS convention: +Extension - Flexion, +ulnar dev -radial dev
# 7/28 discovered wrong flipping for certain tegus; all opossums fine and just need the standard elbow Y,Z flip, but
# don't flip 105 at all
# flip elbowz elbowy for everyone else
# additionally flip glenoidx glenoidz for 108R
jcs_concat.loc[(jcs_concat['animal']!='105'),['elbow.ty','elbow.tz','elbow.ry','elbow.rz']] *= -1
jcs_concat.loc[(jcs_concat['animal']=='108')&(jcs_concat['side']=='R'),['glenoid.tx','glenoid.tz','glenoid.rx','glenoid.rz']] *= -1

rob_df = jcs_concat.copy()
#flip left and right labels for Rob's version
rob_df.replace({'side':'R'}, 'Left', inplace=True)
rob_df.replace({'side':'L'}, 'Right', inplace=True)
rob_df.to_csv('/Users/phil/Downloads/ex_vivo_rob.csv')

         
raw_muscle_names = getMuscleNames(ma_dfs)
alias_map = getAliases(aliases, raw_muscle_names)
reverse_alias_map = reverseAliasMap(alias_map)
ma_dfs = renameMusclesWithMap(ma_dfs, reverse_alias_map)
ma_dfs = generateNewCols(ma_dfs, mode='mma')
#merge moment arm data
ma_concat = pd.concat(ma_dfs.values())


all_data = pd.merge(jcs_concat, ma_concat)

#scale moment arms and translations
re_t = '(.t[xyz])'
t_columns = [col for col in all_data.columns if re.search( re_t, col)]
ma_columns = [col for col in all_data.columns if col[-2:] == 'ma']
t_ma_all_data = all_data[t_columns+ma_columns]
# scaled_t_ma_all_data = t_ma_all_data.divide(all_data['humeralVolCubeRoot'],axis=0)
scaled_t_ma_all_data = t_ma_all_data.divide(all_data['humeralLength'],axis=0).multiply(all_data['humeralLengthFactor'],axis=0)

scaled_t_ma_all_data = scaled_t_ma_all_data.add_prefix('scaled_')

#cosine-correct rotations
jcs_alldata = all_data.filter(like='.r')
##old wrong
# jcs_alldata['radRy'] = np.radians(jcs_alldata['Ry'])
# jcs_alldata['cosRy'] = np.cos(jcs_alldata['radRy'])
# jcs_alldata['RxcosRy'] = jcs_alldata['cosRy'].multiply(jcs_alldata['Rx'])
# all_data['cRx'] = jcs_alldata['RxcosRy']

jcs_alldata.loc[:,'elbow.radRy'] = np.radians(jcs_alldata['elbow.ry'])
jcs_alldata.loc[:,'glenoid.radRy'] = np.radians(jcs_alldata['glenoid.ry'])
jcs_alldata.loc[:,'acromion.radRy'] = np.radians(jcs_alldata['acromion.ry'])


jcs_alldata.loc[:,'elbow.cosRy'] = np.cos(jcs_alldata['elbow.radRy'])
jcs_alldata.loc[:,'glenoid.cosRy'] = np.cos(jcs_alldata['glenoid.radRy'])
jcs_alldata.loc[:,'acromion.cosRy'] = np.cos(jcs_alldata['acromion.radRy'])


jcs_alldata.loc[:,'elbow.RzcosRy'] = jcs_alldata['elbow.cosRy'].multiply(jcs_alldata['elbow.rz'])
jcs_alldata.loc[:,'glenoid.RzcosRy'] = jcs_alldata['glenoid.cosRy'].multiply(jcs_alldata['glenoid.rz'])
jcs_alldata.loc[:,'acromion.RzcosRy'] = jcs_alldata['acromion.cosRy'].multiply(jcs_alldata['acromion.rz'])


all_data.loc[:,'glenoid.crz'] = jcs_alldata['glenoid.RzcosRy']
all_data.loc[:,'elbow.crz'] = jcs_alldata['elbow.RzcosRy']
all_data.loc[:,'acromion.crz'] = jcs_alldata['acromion.RzcosRy']


df = all_data.join(scaled_t_ma_all_data)


#reverse left-side translations (labeled right-side in data imports due to XROMM mirroring). This assumes axes are correctly flipped and now follow protocol
new_t_columns = [col for col in df.columns if re.search( re_t, col)]
df.loc[df['side']=='R',new_t_columns] *= -1


# col_order = ['Tx', 'Ty', 'Tz', 'Rx', 'Ry', 'Rz', 'cRz', frame', 'animal', 'side','species', 'humeralLengthMm']+sorted(t_columns)+sorted(ma_columns)
# df = df[col_order]

#randomly undersample opossum data to match tegu n
# numerical_cols = df.select_dtypes(include='float').reset_index(level=0, inplace=False)
# numerical_cols_nona = numerical_cols.fillna(0)
# rus = RandomUnderSampler(sampling_strategy=1)
# X_resampled, y_resampled = rus.fit_resample(numerical_cols_nona, df['species'])
# df = df.iloc[X_resampled['index']].sort_index()

dfS = df[df['species']=='Tegu']
dfD = df[df['species']=='Opossum']

# drop outliers where z-score > 4
z_D = np.abs(stats.zscore(dfD.filter(like='.r')))
z_S = np.abs(stats.zscore(dfS.filter(like='.r')))
# joint_outliers = dfD.iloc[np.where(z_D>4)[0]].index.union(dfS.iloc[np.where(z_S>4)[0]].index)
# df = df.drop(joint_outliers)
dfS = df[df['species']=='Tegu']
dfD = df[df['species']=='Opossum']

/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/usr/local/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row

### Check relationship between proretraction and crancaudal translation of glenohumeral
pretty strong correlation if we throw out 108, also tz~ry

In [ ]:
dfSforCorr = dfS.filter(regex='glenoid')
dfSforCorr = dfSforCorr.dropna(axis=1, how='all')
corr = dfSforCorr.corr()
corr.style.background_gradient(cmap='bwr')


### Get experimental data ranges

In [ ]:
def roundUpToNearest(value, nearest):
    modulo = value%nearest
    if modulo == 0:
        result = value
    elif value<0:
        result = value-modulo
    else:          
        result = value-value%nearest+nearest
    return result

#calculate ranges
def getRotationRanges(df1, df2, joint, toNearest=None):
    maxrx = math.ceil(max([max(df1[joint+'.rx']),max(df2[joint+'.rx'])])/10)*10
    maxcrz = math.ceil(max([max(df1[joint+'.crz']),max(df2[joint+'.crz'])])/10)*10
    minrx = math.floor(min([min(df1[joint+'.rx']),min(df2[joint+'.rx'])])/10)*10
    mincrz = math.floor(min([min(df1[joint+'.crz']),min(df2[joint+'.crz'])])/10)*10
    maxry = math.ceil(max([max(df1[joint+'.ry']),max(df2[joint+'.ry'])])/10)*10
    minry = math.floor(min([min(df1[joint+'.ry']),min(df2[joint+'.ry'])])/10)*10
    maxrz = math.ceil(max([max(df1[joint+'.rz']),max(df2[joint+'.rz'])])/10)*10
    minrz = math.floor(min([min(df1[joint+'.rz']),min(df2[joint+'.rz'])])/10)*10
    result = {'maxrx':maxrx, 'maxcrz':maxcrz, 'minrx':minrx, 'mincrz':mincrz, 'maxry':maxry, 'minry':minry, 'maxrz':maxrz, 'minrz':minrz}
    if toNearest:
        result = {key: roundUpToNearest(val, toNearest) for key,val in result.items() }
    return result

def getTranslationRanges(df1, df2, joint, toNearest=None):
    maxtx = math.ceil(max([max(df1[joint+'.tx']),max(df2[joint+'.tx'])])/.1)*.1
    mintx = math.floor(min([min(df1[joint+'.tx']),min(df2[joint+'.tx'])])/.1)*.1
    maxty = math.ceil(max([max(df1[joint+'.ty']),max(df2[joint+'.ty'])])/.1)*.1
    minty = math.floor(min([min(df1[joint+'.ty']),min(df2[joint+'.ty'])])/.1)*.1
    maxtz = math.ceil(max([max(df1[joint+'.tz']),max(df2[joint+'.tz'])])/.1)*.1
    mintz = math.floor(min([min(df1[joint+'.tz']),min(df2[joint+'.tz'])])/.1)*.1
    result = {'maxtx':maxtx, 'mintx':mintx, 'maxty':maxty, 'minty':minty, 'maxtz':maxtz, 'mintz':mintz}
    if toNearest:
        result = {key: round(roundUpToNearest(val, toNearest),2) for key,val in result.items() }
    return result

scap_ranges = getRotationRanges(dfS, dfD, 'acromion',30)
glenoid_ranges = getRotationRanges(dfS, dfD, 'glenoid',30)
elbow_ranges = getRotationRanges(dfS, dfD, 'elbow',30)

# maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = rotations.values()

scap_ranges_t = getTranslationRanges(dfS, dfD, 'acromion', .5)
glenoid_ranges_t = getTranslationRanges(dfS, dfD, 'glenoid', .5)
elbow_ranges_t = getTranslationRanges(dfS, dfD, 'elbow', .5)


#find min, max frames
sMaxrx_glenoid = dfS[dfS['glenoid.rx']==max(dfS['glenoid.rx'])]
sMinrx_glenoid = dfS[dfS['glenoid.rx']==min(dfS['glenoid.rx'])]
sMaxry_glenoid = dfS[dfS['glenoid.ry']==max(dfS['glenoid.ry'])]
sMinry_glenoid = dfS[dfS['glenoid.ry']==min(dfS['glenoid.ry'])]
sMaxcrz_glenoid = dfS[dfS['glenoid.crz']==max(dfS['glenoid.crz'])]
sMincrz_glenoid = dfS[dfS['glenoid.crz']==min(dfS['glenoid.crz'])]
sMaxrz_glenoid = dfS[dfS['glenoid.rz']==max(dfS['glenoid.rz'])]
sMinrz_glenoid = dfS[dfS['glenoid.rz']==min(dfS['glenoid.rz'])]

sMaxrx_elbow = dfS[dfS['elbow.rx']==max(dfS['elbow.rx'])]
sMinrx_elbow = dfS[dfS['elbow.rx']==min(dfS['elbow.rx'])]
sMaxry_elbow = dfS[dfS['elbow.ry']==max(dfS['elbow.ry'])]
sMinry_elbow = dfS[dfS['elbow.ry']==min(dfS['elbow.ry'])]
sMaxcrz_elbow = dfS[dfS['elbow.crz']==max(dfS['elbow.crz'])]
sMincrz_elbow = dfS[dfS['elbow.crz']==min(dfS['elbow.crz'])]
sMaxrz_elbow = dfS[dfS['elbow.rz']==max(dfS['elbow.rz'])]
sMinrz_elbow = dfS[dfS['elbow.rz']==min(dfS['elbow.rz'])]

dMaxrx_glenoid = dfD[dfD['glenoid.rx']==max(dfD['glenoid.rx'])]
dMinrx_glenoid = dfD[dfD['glenoid.rx']==min(dfD['glenoid.rx'])]
dMaxry_glenoid = dfD[dfD['glenoid.ry']==max(dfD['glenoid.ry'])]
dMinry_glenoid = dfD[dfD['glenoid.ry']==min(dfD['glenoid.ry'])]
dMaxcrz_glenoid = dfD[dfD['glenoid.crz']==max(dfD['glenoid.crz'])]
dMincrz_glenoid = dfD[dfD['glenoid.crz']==min(dfD['glenoid.crz'])]
dMaxrz_glenoid = dfD[dfD['glenoid.rz']==max(dfD['glenoid.rz'])]
dMinrz_glenoid = dfD[dfD['glenoid.rz']==min(dfD['glenoid.rz'])]

dMaxrx_elbow = dfD[dfD['elbow.rx']==max(dfD['elbow.rx'])]
dMinrx_elbow = dfD[dfD['elbow.rx']==min(dfD['elbow.rx'])]
dMaxry_elbow = dfD[dfD['elbow.ry']==max(dfD['elbow.ry'])]
dMinry_elbow = dfD[dfD['elbow.ry']==min(dfD['elbow.ry'])]
dMaxcrz_elbow = dfD[dfD['elbow.crz']==max(dfD['elbow.crz'])]
dMincrz_elbow = dfD[dfD['elbow.crz']==min(dfD['elbow.crz'])]
dMaxrz_elbow = dfD[dfD['elbow.rz']==max(dfD['elbow.rz'])]
dMinrz_elbow = dfD[dfD['elbow.rz']==min(dfD['elbow.rz'])]

## Plots
### (OLD)Experimental ROM 6DOF

In [ ]:
def Rgb6DOF(df, species, joint, xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.replace(np.NaN, 0)
    df = df.filter(like=joint+'.t')
    tX = df.filter(like=joint+'.tx')
    tY = df.filter(like=joint+'.ty')
    tZ = df.filter(like=joint+'.tz')
    pos_tX = np.where(tX > 0, tX, 0)
    pos_tY = np.where(tY > 0, tY, 0)
    pos_tZ = np.where(tZ > 0, tZ, 0)
    neg_tX = np.where(tX < 0, abs(tX), 0)
    neg_tY = np.where(tY < 0, abs(tY), 0)
    neg_tZ = np.where(tZ < 0, abs(tZ), 0)
    df=df.drop([tX.columns[0],tY.columns[0],tZ.columns[0]], axis=1)
    tMax = df.max().max()
    df=df/tMax
    print(pos_tX)
    df['tX'] = pos_tX if xPos else neg_tX
    df['tY'] = pos_tY if yPos else neg_tY
    df['tZ'] = pos_tZ if zPos else neg_tZ
    totalXYZ = df[['tX','tY','tZ']].sum(axis=1)
    df['scale'] = preprocessing.MinMaxScaler().fit_transform(totalXYZ.values.reshape(-1, 1))
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species]

    return(df[['tX','tY','tZ','scale']])

dfS_6dofPos = Rgb6DOF(df, 'Tegu', 'glenoid', xPos=True, yPos=True, zPos=True)
dfD_6dofPos = Rgb6DOF(df, 'Opossum', 'glenoid', xPos=True, yPos=True, zPos=True)
dfS_6dofNeg = Rgb6DOF(df, 'Tegu', 'glenoid', xPos=False, yPos=False, zPos=False)
dfD_6dofNeg = Rgb6DOF(df, 'Opossum', 'glenoid', xPos=False, yPos=False, zPos=False)


startView = viewXZ


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = glenoid_ranges.values()


plt.close('all')


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(221, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(222, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(223, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(224, projection='3d', proj_type = 'ortho')



format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)
format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=startView)

addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=dfS_6dofPos['scale']*25, c=dfS_6dofPos[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=dfD_6dofPos['scale']*25, c=dfD_6dofPos[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=dfS_6dofNeg['scale']*25, c=dfS_6dofNeg[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=dfD_6dofNeg['scale']*25, c=dfD_6dofNeg[['tX','tY','tZ']], vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection4 = fig.canvas.mpl_connect('motion_notify_event', on_move4)

plt.show()



### In vivo ROM vs ex vivo ROM

### Import in vivo data

In [ ]:
df_invivo = pd.read_csv('./invivo_for_exvivo_comparison.csv',index_col=None).filter(regex='uid|frame|species|individual|condition|glenoid.rotate|elbow.rotate|run')
df_invivo.columns = ['percentage','frame','species','individual','condition','glenoid.rx', 'glenoid.ry', 'glenoid.rz', 'elbow.rx', 'elbow.ry', 'elbow.rz']
df_invivo['percentage'] = df_invivo['percentage'].str.rsplit('_',1).str[-1]
glenoid_radRy = np.radians(df_invivo['glenoid.ry'])
elbow_radRy = np.radians(df_invivo['glenoid.ry'])
glenoid_cosRy = np.cos(glenoid_radRy)
elbow_cosRy = np.cos(elbow_radRy)
df_invivo['glenoid.crz'] = glenoid_cosRy.multiply(df_invivo['glenoid.rz'])
df_invivo['elbow.crz'] = elbow_cosRy.multiply(df_invivo['elbow.rz'])

df_invivo_stand = df_invivo.loc[df_invivo['condition']=='stand']
df_invivo_walk = df_invivo.loc[df_invivo['condition']!='stand']

df_invivo_S = df_invivo_walk.loc[df_invivo_walk['species']=='Tegu']
df_invivo_D = df_invivo_walk.loc[df_invivo_walk['species']=='Opossum']


### Calculate alpha shapes

In [ ]:

alphaVertScap_S,alphaEdgeScap_S,alphaTriScap_S = alpha_shape_3D(dfS.loc[:,['acromion.rx','acromion.ry','acromion.crz']].to_numpy(),20)
alphaVertScap_D,alphaEdgeScap_D,alphaTriScap_D = alpha_shape_3D(dfD.loc[:,['acromion.rx','acromion.ry','acromion.crz']].to_numpy(),20)
alphaVertShoulder_S,alphaEdgeShoulder_S,alphaTriShoulder_S = alpha_shape_3D(dfS.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),20)
alphaVertShoulder_D,alphaEdgeShoulder_D,alphaTriShoulder_D = alpha_shape_3D(dfD.loc[:,['glenoid.rx','glenoid.ry','glenoid.crz']].to_numpy(),20)
alphaVertElbow_S,alphaEdgeElbow_S,alphaTriElbow_S = alpha_shape_3D(dfS.loc[:,['elbow.rx','elbow.ry','elbow.crz']].to_numpy(),20)
alphaVertElbow_D,alphaEdgeElbow_D,alphaTriElbow_D = alpha_shape_3D(dfD.loc[:,['elbow.rx','elbow.ry','elbow.crz']].to_numpy(),20)



alphaVertScap_S_translation,alphaEdgeScap_S_translation,alphaTriScap_S_translation = alpha_shape_3D(dfS.loc[:,['acromion.tx','acromion.ty','acromion.tz']].to_numpy(),20)
alphaVertScap_D_translation,alphaEdgeScap_D_translation,alphaTriScap_D_translation = alpha_shape_3D(dfD.loc[:,['acromion.tx','acromion.ty','acromion.tz']].to_numpy(),20)
alphaVertShoulder_S_translation,alphaEdgeShoulder_S_translation,alphaTriShoulder_S_translation = alpha_shape_3D(dfS.loc[:,['glenoid.tx','glenoid.ty','glenoid.tz']].to_numpy(),20)
alphaVertShoulder_D_translation,alphaEdgeShoulder_D_translation,alphaTriShoulder_D_translation = alpha_shape_3D(dfD.loc[:,['glenoid.tx','glenoid.ty','glenoid.tz']].to_numpy(),20)
alphaVertElbow_S_translation,alphaEdgeElbow_S_translation,alphaTriElbow_S_translation = alpha_shape_3D(dfS.loc[:,['elbow.tx','elbow.ty','elbow.tz']].to_numpy(),20)
alphaVertElbow_D_translation,alphaEdgeElbow_D_translation,alphaTriElbow_D_translation = alpha_shape_3D(dfD.loc[:,['elbow.tx','elbow.ty','elbow.tz']].to_numpy(),20)


In [ ]:

startView = viewXY



glenoid_ranges = getRotationRanges(dfS, dfD, 'glenoid',30)
elbow_ranges = getRotationRanges(dfS, dfD, 'elbow',30)
comp_ranges = {key: max(glenoid_ranges[key], elbow_ranges[key]) for key in glenoid_ranges}
maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = comp_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(3,4,1, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(3,4,2, projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(3,4,5, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(3,4,6, projection='3d', proj_type = 'ortho')

ax4 = fig.add_subplot(3,4,9, projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(3,4,10, projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(3,4,3, projection='3d', proj_type = 'ortho')
ax7 = fig.add_subplot(3,4,4, projection='3d', proj_type = 'ortho')

ax8 = fig.add_subplot(3,4,7, projection='3d', proj_type = 'ortho')
ax9 = fig.add_subplot(3,4,8, projection='3d', proj_type = 'ortho')

ax10 = fig.add_subplot(3,4,11, projection='3d', proj_type = 'ortho')
ax11 = fig.add_subplot(3,4,12, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu glenoid', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, 'Opossum glenoid', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax6, 'Tegu elbow', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
format3dPlot(ax7, 'Opossum elbow', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
format3dPlot(ax8, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')
format3dPlot(ax9, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')
format3dPlot(ax10, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
format3dPlot(ax11, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')

addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)


ax0.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax1.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax2.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax3.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax4.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax5.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)

ax6.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax7.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax8.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax9.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)
ax10.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True, alpha=0.5)
ax11.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True, alpha=0.5)

good_tegu = df_invivo_S

# .loc[((df_invivo_S['individual']=='Tegu 2')&(df_invivo_S['condition']=='hi'))]


df_invivo_S_filt = good_tegu
df_invivo_D_filt = df_invivo_D.loc[(df_invivo_D['condition']=='hi')]

#.loc[(df_invivo_S['individual']!='Tegu 3')&(df_invivo_S['condition']!='lo')]
# df_invivo_S_filt = df_invivo_S.loc[(df_invivo_S['percentage'].astype('int')>40)]
# df_invivo_D_filt = df_invivo_D.loc[(df_invivo_D['individual']!='Opossum 3')&(df_invivo_D['condition']!='lo')]


# .loc[(df_invivo_S['individual']=='Tegu 3')&(df_invivo_S['condition']=='lo')]


# df_invivo_D = df_invivo_D.loc[(df_invivo_D['individual']!='Opossum 3')&(df_invivo_D['condition']!='lo')]


ax0.scatter(df_invivo_S_filt['glenoid.rx'],df_invivo_S_filt['glenoid.ry'],df_invivo_S_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax1.scatter(df_invivo_D_filt['glenoid.rx'],df_invivo_D_filt['glenoid.ry'],df_invivo_D_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax2.scatter(df_invivo_S_filt['glenoid.rx'],df_invivo_S_filt['glenoid.ry'],df_invivo_S_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax3.scatter(df_invivo_D_filt['glenoid.rx'],df_invivo_D_filt['glenoid.ry'],df_invivo_D_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax4.scatter(df_invivo_S_filt['glenoid.rx'],df_invivo_S_filt['glenoid.ry'],df_invivo_S_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax5.scatter(df_invivo_D_filt['glenoid.rx'],df_invivo_D_filt['glenoid.ry'],df_invivo_D_filt['glenoid.crz'], s=0.5, depthshade=True, color='black', zorder=3)

ax6.scatter(df_invivo_S_filt['elbow.rx'],df_invivo_S_filt['elbow.ry'],df_invivo_S_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax7.scatter(df_invivo_D_filt['elbow.rx'],df_invivo_D_filt['elbow.ry'],df_invivo_D_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax8.scatter(df_invivo_S_filt['elbow.rx'],df_invivo_S_filt['elbow.ry'],df_invivo_S_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax9.scatter(df_invivo_D_filt['elbow.rx'],df_invivo_D_filt['elbow.ry'],df_invivo_D_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax10.scatter(df_invivo_S_filt['elbow.rx'],df_invivo_S_filt['elbow.ry'],df_invivo_S_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)
ax11.scatter(df_invivo_D_filt['elbow.rx'],df_invivo_D_filt['elbow.ry'],df_invivo_D_filt['elbow.crz'], s=0.5, depthshade=True, color='black', zorder=3)

# connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)



### DONT USE: FIGURE: ROM envelopes: combined 

In [ ]:
# scatterSize=0.15
# scatterAlpha=0.1
# scatterTeal = '#3CA9A6'
# scatterGold = '#FFCA4E'

# shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }
# combined_ranges = {key: (shoulderElbow_ranges[key] if (abs(shoulderElbow_ranges[key])>abs(scap_ranges[key])) else scap_ranges[key])  for key in shoulderElbow_ranges.keys() }


# shoulderElbow_ranges_translation = {key: (elbow_ranges_t[key] if (abs(elbow_ranges_t[key])>abs(glenoid_ranges_t[key])) else glenoid_ranges_t[key])  for key in elbow_ranges_t.keys() }
# combined_ranges_translation = {key: (shoulderElbow_ranges_translation[key] if (abs(shoulderElbow_ranges_translation[key])>abs(scap_ranges_t[key])) else scap_ranges_t[key])  for key in shoulderElbow_ranges_translation.keys() }


# maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = combined_ranges.values()

# maxtx, mintx, maxty, minty, maxtz, mintz = combined_ranges_translation.values()


# plt.close('all')
# plt.rcParams['grid.linewidth'] = 0
# plt.rcParams['grid.color'] = 'lightgrey'

# fig = plt.figure(figsize=[18,10], constrained_layout=True)

# gs3col = gridspec.GridSpec(1,3, figure=fig)
# gs3row_scap = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs3col[0])
# gs3row_glen = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs3col[1])
# gs3row_elb = gridspec.GridSpecFromSubplotSpec(3, 1, subplot_spec=gs3col[2])

# gs2col_scap_YZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_scap[0])
# gs2col_scap_YX = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_scap[1])
# gs2col_scap_XZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_scap[2])

# gs2col_glen_YZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_glen[0])
# gs2col_glen_YX = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_glen[1])
# gs2col_glen_XZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_glen[2])

# gs2col_elb_YZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_elb[0])
# gs2col_elb_YX = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_elb[1])
# gs2col_elb_XZ = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=gs3row_elb[2])

# ax_scap_YZ_S = fig.add_subplot(gs2col_scap_YZ[0], projection='3d', proj_type = 'ortho')
# ax_scap_YX_S = fig.add_subplot(gs2col_scap_YX[0], projection='3d', proj_type = 'ortho')
# ax_scap_XZ_S = fig.add_subplot(gs2col_scap_XZ[0], projection='3d', proj_type = 'ortho')

# ax_scap_YZ_D = fig.add_subplot(gs2col_scap_YZ[1], projection='3d', proj_type = 'ortho')
# ax_scap_YX_D = fig.add_subplot(gs2col_scap_YX[1], projection='3d', proj_type = 'ortho')
# ax_scap_XZ_D = fig.add_subplot(gs2col_scap_XZ[1], projection='3d', proj_type = 'ortho')

# ax_glen_YZ_S = fig.add_subplot(gs2col_glen_YZ[0], projection='3d', proj_type = 'ortho')
# ax_glen_YX_S = fig.add_subplot(gs2col_glen_YX[0], projection='3d', proj_type = 'ortho')
# ax_glen_XZ_S = fig.add_subplot(gs2col_glen_XZ[0], projection='3d', proj_type = 'ortho')

# ax_glen_YZ_D = fig.add_subplot(gs2col_glen_YZ[1], projection='3d', proj_type = 'ortho')
# ax_glen_YX_D = fig.add_subplot(gs2col_glen_YX[1], projection='3d', proj_type = 'ortho')
# ax_glen_XZ_D = fig.add_subplot(gs2col_glen_XZ[1], projection='3d', proj_type = 'ortho')

# ax_elb_YZ_S = fig.add_subplot(gs2col_elb_YZ[0], projection='3d', proj_type = 'ortho')
# ax_elb_YX_S = fig.add_subplot(gs2col_elb_YX[0], projection='3d', proj_type = 'ortho')
# ax_elb_XZ_S = fig.add_subplot(gs2col_elb_XZ[0], projection='3d', proj_type = 'ortho')

# ax_elb_YZ_D = fig.add_subplot(gs2col_elb_YZ[1], projection='3d', proj_type = 'ortho')
# ax_elb_YX_D = fig.add_subplot(gs2col_elb_YX[1], projection='3d', proj_type = 'ortho')
# ax_elb_XZ_D = fig.add_subplot(gs2col_elb_XZ[1], projection='3d', proj_type = 'ortho')


# # tax_scap_YZ_S = fig.add_subplot(gs_t[0,0], projection='3d', proj_type = 'ortho')
# # tax_scap_YX_S = fig.add_subplot(gs_t[0,2], projection='3d', proj_type = 'ortho')
# # tax_scap_XZ_S = fig.add_subplot(gs_t[0,1], projection='3d', proj_type = 'ortho')

# # tax_scap_YZ_D = fig.add_subplot(gs_t[1,0], projection='3d', proj_type = 'ortho')
# # tax_scap_YX_D = fig.add_subplot(gs_t[1,2], projection='3d', proj_type = 'ortho')
# # tax_scap_XZ_D = fig.add_subplot(gs_t[1,1], projection='3d', proj_type = 'ortho')

# # tax_glen_YZ_S = fig.add_subplot(gs_t[2,0], projection='3d', proj_type = 'ortho')
# # tax_glen_YX_S = fig.add_subplot(gs_t[2,2], projection='3d', proj_type = 'ortho')
# # tax_glen_XZ_S = fig.add_subplot(gs_t[2,1], projection='3d', proj_type = 'ortho')

# # tax_glen_YZ_D = fig.add_subplot(gs_t[3,0], projection='3d', proj_type = 'ortho')
# # tax_glen_YX_D = fig.add_subplot(gs_t[3,2], projection='3d', proj_type = 'ortho')
# # tax_glen_XZ_D = fig.add_subplot(gs_t[3,1], projection='3d', proj_type = 'ortho')

# # tax_elb_YZ_S = fig.add_subplot(gs_t[4,0], projection='3d', proj_type = 'ortho')
# # tax_elb_YX_S = fig.add_subplot(gs_t[4,2], projection='3d', proj_type = 'ortho')
# # tax_elb_XZ_S = fig.add_subplot(gs_t[4,1], projection='3d', proj_type = 'ortho')

# # tax_elb_YZ_D = fig.add_subplot(gs_t[5,0], projection='3d', proj_type = 'ortho')
# # tax_elb_YX_D = fig.add_subplot(gs_t[5,2], projection='3d', proj_type = 'ortho')
# # tax_elb_XZ_D = fig.add_subplot(gs_t[5,1], projection='3d', proj_type = 'ortho')

# # format3dPlot(tax_scap_YZ_S, '', (mintx, maxtx), (minty, maxty), (mintz, maxtz), view=viewYZ, mode='translations',joint='scap', tickInterval=0.5)


# format3dPlot(ax_scap_YZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='scap')
# format3dPlot(ax_scap_YZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='scap')
# format3dPlot(ax_glen_YZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='glenoid')
# format3dPlot(ax_glen_YZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='glenoid')
# format3dPlot(ax_elb_YZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
# format3dPlot(ax_elb_YZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')

# format3dPlot(ax_scap_YX_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='scap')
# format3dPlot(ax_scap_YX_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='scap')
# format3dPlot(ax_glen_YX_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='glenoid')
# format3dPlot(ax_glen_YX_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='glenoid')
# format3dPlot(ax_elb_YX_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='elbow')
# format3dPlot(ax_elb_YX_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYX, joint='elbow')

# format3dPlot(ax_scap_XZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='scap')
# format3dPlot(ax_scap_XZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='scap')
# format3dPlot(ax_glen_XZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='glenoid')
# format3dPlot(ax_glen_XZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='glenoid')
# format3dPlot(ax_elb_XZ_S, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
# format3dPlot(ax_elb_XZ_D, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')


# addCosGrid(ax_scap_YZ_S, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_scap_YZ_D, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_glen_YZ_S, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_glen_YZ_D, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_elb_YZ_S, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
# addCosGrid(ax_elb_YZ_D, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)


# ax_scap_YZ_S.plot_trisurf(dfS['acromion.rx'],dfS['acromion.ry'],alphaTriScap_S,dfS['acromion.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_scap_YX_S.plot_trisurf(dfS['acromion.rx'],dfS['acromion.ry'],alphaTriScap_S,dfS['acromion.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_scap_XZ_S.plot_trisurf(dfS['acromion.rx'],dfS['acromion.ry'],alphaTriScap_S,dfS['acromion.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_scap_YZ_D.plot_trisurf(dfD['acromion.rx'],dfD['acromion.ry'],alphaTriScap_D,dfD['acromion.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_scap_YX_D.plot_trisurf(dfD['acromion.rx'],dfD['acromion.ry'],alphaTriScap_D,dfD['acromion.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_scap_XZ_D.plot_trisurf(dfD['acromion.rx'],dfD['acromion.ry'],alphaTriScap_D,dfD['acromion.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)

# ax_glen_YZ_S.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_glen_YX_S.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_glen_XZ_S.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_glen_YZ_D.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_glen_YX_D.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_glen_XZ_D.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)

# ax_elb_YZ_S.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_elb_YX_S.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_elb_XZ_S.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax_elb_YZ_D.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_elb_YX_D.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)
# ax_elb_XZ_D.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd',linewidth=0, antialiased=True)

# ax_scap_YZ_S.set_title('Tegu')
# ax_glen_YZ_S.set_title('Tegu')
# ax_elb_YZ_S.set_title('Tegu')
# ax_scap_YZ_D.set_title('Opossum')
# ax_glen_YZ_D.set_title('Opossum')
# ax_elb_YZ_D.set_title('Opossum')

# ax_scap_YZ_S.scatter(dfS['acromion.rx'],dfS['acromion.ry'],dfS['acromion.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_YX_S.scatter(dfS['acromion.rx'],dfS['acromion.ry'],dfS['acromion.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_XZ_S.scatter(dfS['acromion.rx'],dfS['acromion.ry'],dfS['acromion.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_YZ_D.scatter(dfD['acromion.rx'],dfD['acromion.ry'],dfD['acromion.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_YX_D.scatter(dfD['acromion.rx'],dfD['acromion.ry'],dfD['acromion.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_scap_XZ_D.scatter(dfD['acromion.rx'],dfD['acromion.ry'],dfD['acromion.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)

# ax_glen_YZ_S.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_YX_S.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_XZ_S.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_YZ_D.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_YX_D.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_glen_XZ_D.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)

# ax_elb_YZ_S.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_YX_S.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_XZ_S.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=scatterSize, c=scatterTeal, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_YZ_D.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_YX_D.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)
# ax_elb_XZ_D.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=scatterSize, c=scatterGold, depthshade=True, zorder=4, alpha=scatterAlpha)



# # connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# # def animate(i):
# #     ax0.view_init(elev=45, azim=i/10)
# #     ax1.view_init(elev=45, azim=i/10)
# #     return fig

# # both_ani = animation.FuncAnimation(fig, animate, 
# #                                frames=3600, interval=5, blit=True)

# # plt.show()

# # plt.savefig('both3Q.png', dpi=150, transparent=True)


### FIGURE: ROM envelopes: glenoid big 3q
rob's data cuts off at + - 90 degrees from neutral, which is 0,0,-20 for tegu and 0, -40, -80 for opossum

In [ ]:
smallScatter = 0.25
bigScatter = 0.75

shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

dfS_robCrop = dfS.loc[(dfS['glenoid.rx'].between(-90,90))&(dfS['glenoid.ry'].between(-90,90))&(dfS['glenoid.rz'].between(-110,70))]
dfD_robCrop = dfD.loc[(dfD['glenoid.rx'].between(-90,90))&(dfD['glenoid.ry'].between(-130,50))&(dfD['glenoid.rz'].between(-170,10))]

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/rotations_base.eps', dpi=300, transparent=True)




addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)



ax0.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax1.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax2.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax3.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
ax4.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax5.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax6.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTriShoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax7.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTriShoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=bigScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=bigScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7]:
    axis.axis('off')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/glenoid_rotations.png', dpi=300, transparent=True)


In [ ]:
dfS.loc[dfS['glenoid.rx']>50]['glenoid.crz']

### FIGURE: ROM envelopes:elbow big 3q

In [ ]:
smallScatter = 0.25
bigScatter = 0.75

shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ, joint='elbow')

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ, joint='elbow')
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY, joint='elbow')

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos, joint='elbow')
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos, joint='elbow')

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/rotations_base_elbow.eps', dpi=300, transparent=True)


addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)



ax0.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax1.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax2.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax3.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
ax4.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax5.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

ax6.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
ax7.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=smallScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=smallScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=bigScatter, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=bigScatter, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

for axis in [ax0, ax1, ax2, ax3, ax4, ax5, ax6, ax7]:
    axis.axis('off')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

plt.savefig('/Users/phil/Desktop/rotoscoping/Figures/ch3plots/elbow_rotations.png', dpi=300, transparent=True)


### test plots these dont save

In [ ]:


startView = viewXY
shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

dfS_robCrop = dfS.loc[(dfS['glenoid.rx'].between(-90,90))&(dfS['glenoid.ry'].between(-90,90))&(dfS['glenoid.rz'].between(-110,70))]
dfD_robCrop = dfD.loc[(dfD['glenoid.rx'].between(-90,90))&(dfD['glenoid.ry'].between(-130,50))&(dfD['glenoid.rz'].between(-170,10))]

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)





addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

# ax0.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax1.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax2.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax3.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
# ax4.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax5.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax6.plot_trisurf(dfS['glenoid.rx'],dfS['glenoid.ry'],alphaTrishoulder_S,dfS['glenoid.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax7.plot_trisurf(dfD['glenoid.rx'],dfD['glenoid.ry'],alphaTrishoulder_D,dfD['glenoid.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['glenoid.rx'],dfS['glenoid.ry'],dfS['glenoid.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD['glenoid.rx'],dfD['glenoid.ry'],dfD['glenoid.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)



connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)


In [ ]:
dfD.loc[dfD['glenoid.rx']<-50].filter(like='glenoid.cr')

In [ ]:


startView = viewXY
shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

dfS_robCrop = dfS.loc[(dfS['elbow.rx'].between(-90,90))&(dfS['elbow.ry'].between(-90,90))&(dfS['elbow.rz'].between(-110,70))]
dfD_robCrop = dfD.loc[(dfD['elbow.rx'].between(-90,90))&(dfD['elbow.ry'].between(-130,50))&(dfD['elbow.rz'].between(-170,10))]

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)





addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

# ax0.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax1.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax2.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax3.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
# ax4.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax5.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax6.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax7.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)




ax0.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfD['elbow.rx'],dfD['elbow.ry'],dfD['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)



connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)


In [ ]:
drop 88L 614
drop 101L elbow X : disarticulated in videos: collaterals compromised
85L is fine, actually a lot of supination
drop 108L frame 345: disarticulated

In [ ]:


startView = viewXY
shoulderElbow_ranges = {key: (elbow_ranges[key] if (abs(elbow_ranges[key])>abs(glenoid_ranges[key])) else glenoid_ranges[key])  for key in elbow_ranges.keys() }


maxrx, maxcrz, minrx, mincrz, maxry, minry, maxrz, minrz = shoulderElbow_ranges.values()

plt.close('all')
plt.rcParams['grid.linewidth'] = 0
plt.rcParams['grid.color'] = 'lightgrey'

dfS_robCrop = dfS.loc[(dfS['elbow.rx'].between(-90,90))&(dfS['elbow.ry'].between(-90,90))&(dfS['elbow.rz'].between(-110,70))]
dfD_robCrop = dfD.loc[(dfD['elbow.rx'].between(-90,90))&(dfD['elbow.ry'].between(-130,50))&(dfD['elbow.rz'].between(-170,10))]

fig = plt.figure(figsize=[18,10], constrained_layout=True)
gs = gridspec.GridSpec(6, 6, figure=fig)

ax0 = fig.add_subplot(gs[0:2,2], projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(gs[0:2,3], projection='3d', proj_type = 'ortho')

ax2 = fig.add_subplot(gs[2:4,2], projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(gs[2:4,3], projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(gs[4:6,2], projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(gs[4:6,3], projection='3d', proj_type = 'ortho')

ax6 = fig.add_subplot(gs[0:6,0:2], projection='3d', proj_type = 'ortho')

ax7 = fig.add_subplot(gs[0:6,4:6], projection='3d', proj_type = 'ortho')

format3dPlot(ax0, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)
format3dPlot(ax1, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewYZ)

format3dPlot(ax2, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax3, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXY)
format3dPlot(ax4, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)
format3dPlot(ax5, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=viewXZ)

format3dPlot(ax6, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)
format3dPlot(ax7, '', (minrx, maxrx), (minry, maxry), (mincrz, maxcrz), view=view3Qpos)





addCosGrid(ax0, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax6, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax7, (minrx,maxrx), (minry,maxry), (minrz,maxrz), 5, xLevels=1, color='grey', linewidths=0.1)

# ax0.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax1.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax2.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax3.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)
# ax4.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax5.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)

# ax6.plot_trisurf(dfS['elbow.rx'],dfS['elbow.ry'],alphaTriElbow_S,dfS['elbow.crz'], shade=True, cmap='YlGnBu',linewidth=0, antialiased=True)
# ax7.plot_trisurf(dfD['elbow.rx'],dfD['elbow.ry'],alphaTriElbow_D,dfD['elbow.crz'], shade=True, cmap='YlOrRd', linewidth=0, antialiased=True)


dfDno101L = dfD.loc[~(dfD['animal']=='101')|~(dfD['side']=='L')]

ax0.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax1.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax2.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax3.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)
ax4.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax5.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)

ax6.scatter(dfS['elbow.rx'],dfS['elbow.ry'],dfS['elbow.crz'], s=0.5, c='#3CA9A6', depthshade=True, zorder=4, alpha=.25)
ax7.scatter(dfDno101L['elbow.rx'],dfDno101L['elbow.ry'],dfDno101L['elbow.crz'], s=0.5, c='#FFCA4E', depthshade=True, zorder=4, alpha=.25)



connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

# def animate(i):
#     ax0.view_init(elev=45, azim=i/10)
#     ax1.view_init(elev=45, azim=i/10)
#     return fig

# both_ani = animation.FuncAnimation(fig, animate, 
#                                frames=3600, interval=5, blit=True)

# plt.show()

# plt.savefig('both3Q.png', dpi=150, transparent=True)


In [ ]:
dfD.loc[~(dfD['animal']=='101')&~(dfD['side']=='L')]

In [ ]:
dfD.loc[dfD['elbow.rz']==dfD['elbow.rz'].min()]['frame']

In [ ]:
dfD.loc[(dfD['animal']=='101')&(dfD['side']=='L')]

In [ ]:
dfD

### R shit here

In [ ]:
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri
pandas2ri.activate()
from rpy2.robjects.conversion import localconverter
import rpy2.ipython.html
rpy2.ipython.html.init_printing()
from rpy2.robjects.lib.dplyr import DataFrame
from rpy2.robjects import rl

alphashape3d = importr('alphashape3d')
tidyverse = importr('tidyverse')
ro.r('''
        getCriticalAlpha <- function(components, alphas){
          criticalIndex <- 1
          for (i in 1:length(components)){
            comp = as.numeric(components[[i]])
            if (any(comp != 1)){
              criticalIndex <- i
            }
          }
          criticalIndex <- criticalIndex + 1
          print(alphas[[criticalIndex]])
          return(list(criticalIndex, alphas[[criticalIndex]]))
        }
        ''')
getCriticalAlpha = ro.r['getCriticalAlpha']
with localconverter(ro.default_converter + pandas2ri.converter):
    rdf_S_glenoid = ro.r['as.matrix'](ro.conversion.py2rpy(dfS[['glenoid.rx','glenoid.ry','glenoid.crz']].copy()))
    rdf_D_glenoid = ro.r['as.matrix'](ro.conversion.py2rpy(dfD[['glenoid.rx','glenoid.ry','glenoid.crz']].copy()))
    rdf_S_elbow = ro.r['as.matrix'](ro.conversion.py2rpy(dfS[['elbow.rx','elbow.ry','elbow.crz']].copy()))
    rdf_D_elbow = ro.r['as.matrix'](ro.conversion.py2rpy(dfD[['elbow.rx','elbow.ry','elbow.crz']].copy()))
#     rdf_S_model = ro.r['as.matrix'](ro.conversion.py2rpy(df_M_S[['Rx','Ry','cRz']].copy()))
#     rdf_D_model = ro.r['as.matrix'](ro.conversion.py2rpy(df_M_D[['Rx','Ry','cRz']].copy()))
    
subTenAlphas = np.linspace(0.1,9.9, 99)
supTenAlphas = np.linspace(10, 100, 91)
alphas = np.concatenate((subTenAlphas,supTenAlphas))
alphaCols = [str(round(alpha,3)).replace('.','_') for alpha in alphas]


alphas_sens = np.arange(1,101)
alphaCols_sens = [str(round(alpha,3)).replace('.','_') for alpha in alphas_sens]


def getAlphaObjects(rdf, alphas):
    try:
        alphaCols = [str(round(alpha,3)).replace('.','_') for alpha in alphas]
        print("...calculating alphas...")
        alphaShapes = alphashape3d.ashape3d(rdf, alphas, pert=True)
        print("...calculating components...")
        components = alphashape3d.components_ashape3d(alphaShapes, indexAlpha="all")
        print("...calculating volumes...")
        volumes = alphashape3d.volume_ashape3d(alphaShapes, indexAlpha="all")
    #     print("...calculating normals...")
    #     normals = alphashape3d.surfaceNormals(alphaShapes, indexAlpha="all")
        print("...calculating critical alpha...")
        [crit_index, crit_val] = getCriticalAlpha(components, alphas)
        shapes = np.array(alphaShapes, dtype=object)
        tetras_df = pd.DataFrame(alphaShapes[0]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'index4',4:'intervals'},**dict(zip(range(5,len(alphas)+5),alphaCols))})
        triangles_df = pd.DataFrame(alphaShapes[1]).rename(columns={**{0:'index1',1:'index2',2:'index3',3:'on_convex_hull',4:'attached',5:'intervals1',6:'intervals2',7:'intervals3'},**dict(zip(range(8,len(alphas)+8),alphaCols))})
        edges_df = pd.DataFrame(alphaShapes[2]).rename(columns={**{0:'index1',1:'index2',2:'on_convex_hull',3:'attached',4:'intervals1',5:'intervals2',6:'intervals3'},**dict(zip(range(7,len(alphas)+7),alphaCols))})
        vertices_df = pd.DataFrame(alphaShapes[3]).rename(columns={**{0:'index',1:'on_convex_hull',2:'intervals1',3:'intervals2'},**dict(zip(range(4,len(alphas)+4),alphaCols))})
        vertices_df['x'],vertices_df['y'],vertices_df['z'] = alphaShapes[4].T[0],alphaShapes[4].T[1],alphaShapes[4].T[2]
        components_df = pd.DataFrame(np.array(components, dtype=object).T).rename(columns=dict(zip(range(0,len(alphas)+1),alphaCols)))
        volumes_df = pd.DataFrame(np.array(volumes, dtype=object)).transpose().rename(columns=dict(zip(range(0,len(alphas)+1),alphaCols)))
    #     normals_df = pd.DataFrame(np.array(normals, dtype=object), columns=['normal','center'], index=alphaCols)
        return {'tetrahedrons':tetras_df, 'triangles': triangles_df, 'edges': edges_df, 'vertices': vertices_df,
                'components': components_df, 'volumes': volumes_df, 'crit_val':crit_val, 'crit_index':crit_index }
    except rpy2.rinterface_lib.embedded.RRuntimeError:
        return {}

# alpha_s_glenoid = getAlphaObjects(rdf_S_glenoid, alphas_sens)
# alpha_d_glenoid = getAlphaObjects(rdf_D_glenoid, alphas_sens)
# alpha_s_elbow = getAlphaObjects(rdf_S_elbow, alphas_sens)
# alpha_d_elbow = getAlphaObjects(rdf_D_elbow, alphas_sens)

# %pip install kneed
from kneed import DataGenerator, KneeLocator

def autoAlphaChooser(dfs_with_volumes, names, starting_alpha, sParam=10.0, curve='concave', direction='increasing'):
    volumes_df = pd.concat(dfs_with_volumes, axis=1).astype('float')
    volumes_df.columns=names
    alphas = [float(alphaCol.replace('_','.')) for alphaCol in volumes_df.index]
    maxVol = volumes_df.iloc[-1].max()
    minVol = volumes_df.iloc[-1].min()
    volumes_df.set_index(pd.Index(alphas), inplace=True)
    volumes_df.rename(columns={"index":"alpha"}, inplace=True)
    maxCol = volumes_df.columns[list(volumes_df.iloc[-1].values).index(maxVol)]
    minCol = volumes_df.columns[list(volumes_df.iloc[-1].values).index(minVol)]
    ratio = maxCol+' : '+minCol
    volumes_df[ratio] = np.divide(volumes_df.loc[:,maxCol],volumes_df.loc[:,minCol])
    volumes_df[ratio].fillna(0, inplace=True)
    ratio_series = volumes_df.loc[starting_alpha:,ratio].copy() if starting_alpha else volumes_df.loc[:,ratio].copy()
    kneedle = KneeLocator(list(ratio_series.index), ratio_series, S=sParam, curve=curve, direction=direction)
    return([volumes_df, kneedle])
    
plt.close("all")
volumes_df, auto_alpha = autoAlphaChooser([alpha_s_glenoid['volumes'].T,alpha_d_glenoid['volumes'].T,alpha_s_elbow['volumes'].T,alpha_d_elbow['volumes'].T],['Tegu glenoid','Opossum glenoid','Tegu elbow','Opossum elbow'],15, 10.0, 'convex', 'decreasing')
fig, ax = plt.subplots()
volumes_df.loc[:,['Tegu glenoid','Opossum glenoid','Tegu elbow','Opossum elbow']].plot(ax=ax, color=['C0','C1','#A0CBE8','#F1CE63'])
volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax)
# ax.vlines(auto_alpha.knee, volumes_df.min().min(), volumes_df.max().max(), color='r', label='knee')
# auto_alpha.knee

ax.vlines(20, 0, maxMax, color='red')
ax.annotate(text='20', xy =(((20-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

### Downsampling

In [ ]:
def downsample(df, species, factor=1, random=False):
    full = df.loc[df.species == species].copy()
    full.loc[:,'uid'] = full.index
    full.loc[:,'animalside'] = full.animal+full.side
    if random:
        frames_to_pick = np.sort(np.random.choice(list(full.index), len(full)//factor, replace=False))
        return(full.loc[frames_to_pick])
    for animalside in full.animalside.unique():
        as_df = full[full.animalside == animalside]
        min_frame, max_frame = as_df.frame.min(), as_df.frame.max()
        candidate_range = np.arange(min_frame, max_frame+1)
        frames_to_pick = list(candidate_range[::int(factor)])
        full.drop(full[(full['animalside']==animalside)&(~full['frame'].isin(frames_to_pick))].index, inplace=True)
    return(full)

factors_to_test = np.arange(1,11)
df_S_1to10 = [downsample(df, 'Tegu', factor) for factor in factors_to_test]
df_D_1to10 = [downsample(df, 'Opossum', factor) for factor in factors_to_test]

df_S_1to10_rand = [downsample(df, 'Tegu', factor, random=True) for factor in factors_to_test]
df_D_1to10_rand = [downsample(df, 'Opossum', factor, random=True) for factor in factors_to_test]

In [ ]:
with localconverter(ro.default_converter + pandas2ri.converter):
    rdf_S_glenoid_1to10 = [ro.r['as.matrix'](ro.conversion.py2rpy(dfS[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfS in df_S_1to10]
    rdf_D_glenoid_1to10 = [ro.r['as.matrix'](ro.conversion.py2rpy(dfD[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfD in df_D_1to10]
    rdf_S_glenoid_1to10_rand = [ro.r['as.matrix'](ro.conversion.py2rpy(dfS[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfS in df_S_1to10_rand]
    rdf_D_glenoid_1to10_rand = [ro.r['as.matrix'](ro.conversion.py2rpy(dfD[['glenoid.rx','glenoid.ry','glenoid.crz']].copy())) for dfD in df_D_1to10_rand]

alphas_S_1to10 = [getAlphaObjects(rdf_S_glenoid, alphas_sens) for rdf_S_glenoid in rdf_S_glenoid_1to10]
alphas_D_1to10 = [getAlphaObjects(rdf_D_glenoid, alphas_sens) for rdf_D_glenoid in rdf_D_glenoid_1to10]
alphas_S_1to10_rand = [getAlphaObjects(rdf_S_glenoid_rand, alphas_sens) for rdf_S_glenoid_rand in rdf_S_glenoid_1to10_rand]
alphas_D_1to10_rand = [getAlphaObjects(rdf_D_glenoid_rand, alphas_sens) for rdf_D_glenoid_rand in rdf_D_glenoid_1to10_rand]



import csv
# for i, alphaobj in enumerate(alphas_S_1to10):
#     name = 'alpha_S_'+str(i)
#     with open('./alphas/'+name+'.csv', 'w') as f:
#         for key in alphaobj.keys():
#             f.write("%s,%s\n"%(key,alphaobj[key]))
# for i, alphaobj in enumerate(alphas_D_1to10):
#     name = 'alpha_D_'+str(i)
#     with open('./alphas/'+name+'.csv', 'w') as f:
#         for key in alphaobj.keys():
#             f.write("%s,%s\n"%(key,alphaobj[key]))
for i, alphaobj in enumerate(alphas_S_1to10_rand):
    name = 'alpha_S_rand_'+str(i)
    with open('./alphas/'+name+'.csv', 'w') as f:
        for key in alphaobj.keys():
            f.write("%s,%s\n"%(key,alphaobj[key]))
for i, alphaobj in enumerate(alphas_D_1to10_rand):
    name = 'alpha_D_rand_'+str(i)
    with open('./alphas/'+name+'.csv', 'w') as f:
        for key in alphaobj.keys():
            f.write("%s,%s\n"%(key,alphaobj[key]))

In [ ]:
volumes_df

In [ ]:
plt.close("all")
fig, ax = plt.subplots(3,2, sharey='all', figsize=[15,12])

volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[0]['volumes'].T,alphas_D_1to10[0]['volumes'].T],['Tegu uniform 1/8','Opossum uniform 1/8'],18, 10.0, 'convex', 'decreasing')
volumes_df.loc[:,['Tegu uniform 1/8','Opossum uniform 1/8']].plot(ax=ax[0,0])
volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[0,0])
ax[0,0].vlines(auto_alpha.knee, volumes_df.min().min(), volumes_df.max().max(), color='r')
ax[0,0].title.set_text('1 in 8 (no further downsampling)')
x_bounds = ax[0,0].get_xlim()
ax[0,0].legend(fontsize=10)
ax[0,0].annotate(text=int(auto_alpha.knee), xy =(((auto_alpha.knee-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

yMax = [volumes_df.max().max()]

for v_i, a_i, denom in zip(np.arange(1,6),[(0,1),(1,0),(1,1),(2,0),(2,1)],((np.arange(5)+2)*8).astype('str')):
    volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[v_i]['volumes'].T,alphas_D_1to10[v_i]['volumes'].T],['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df_rand, auto_alpha_rand = autoAlphaChooser([alphas_S_1to10_rand[v_i]['volumes'].T,alphas_D_1to10_rand[v_i]['volumes'].T],['Tegu random 1/'+denom,'Opossum random 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df.loc[:,['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'])
    volumes_df_rand.loc[:,['Tegu random 1/'+denom,'Opossum random 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'], linestyle='dotted')
    volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[a_i])
    volumes_df_rand.loc[:,[volumes_df_rand.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[a_i],linestyle='dotted')
    x_bounds = ax[a_i].get_xlim()
    ax[a_i].legend(fontsize=10)
    ax[a_i].title.set_text('1 in '+denom)
    yMax.append(volumes_df.max().max())

maxMax = max(yMax)

ax[0,0].set_ylabel('Mobility volume (°^3)')
ax[1,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_xlabel('Alpha')
ax[2,1].set_xlabel('Alpha')
ax[0,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[1,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[2,1].right_ax.set_ylabel('Tegu-opossum ratio')


ax[0,1].vlines(22, 0, maxMax, color='r')
ax[0,1].annotate(text='U,R 22', xy =(((22-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

ax[1,0].vlines(24, 0, maxMax, color='r')
ax[1,0].vlines(26, 0, maxMax, color='r', linestyle='dotted')
ax[1,0].annotate(text='U 24', xy =(((24-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,0].annotate(text='R 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[1,1].vlines(26, 0, maxMax, color='r')
ax[1,1].vlines(28, 0, maxMax, color='r', linestyle='dotted')
ax[1,1].annotate(text='U 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,1].annotate(text='R 28', xy =(((28-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,0].vlines(27, 0, maxMax, color='r')
ax[2,0].vlines(30, 0, maxMax, color='r', linestyle='dotted')
ax[2,0].annotate(text='U 27', xy =(((27-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[2,0].annotate(text='R 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,1].vlines(30, 0, maxMax, color='r')
ax[2,1].vlines(37, 0, maxMax, color='r', linestyle='dotted')
ax[2,1].annotate(text='U 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[2,1].annotate(text='R 37', xy =(((37-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)



In [ ]:
plt.close("all")
fig, ax = plt.subplots(3,2, sharey='all', figsize=[15,12])

volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[0]['volumes'].T,alphas_D_1to10[0]['volumes'].T],['Tegu uniform 1/8','Opossum uniform 1/8'],18, 10.0, 'convex', 'decreasing')
volumes_df.loc[:,['Tegu uniform 1/8','Opossum uniform 1/8']].plot(ax=ax[0,0])
volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[0,0])
ax[0,0].vlines(auto_alpha.knee, volumes_df.min().min(), volumes_df.max().max(), color='r')
ax[0,0].title.set_text('1 in 8 (no further downsampling)')
x_bounds = ax[0,0].get_xlim()
ax[0,0].legend(fontsize=10)
ax[0,0].annotate(text=int(auto_alpha.knee), xy =(((auto_alpha.knee-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

yMax = [volumes_df.max().max()]

for v_i, a_i, denom in zip(np.arange(1,6),[(0,1),(1,0),(1,1),(2,0),(2,1)],((np.arange(5)+2)*8).astype('str')):
    volumes_df, auto_alpha = autoAlphaChooser([alphas_S_1to10[v_i]['volumes'].T,alphas_D_1to10[v_i]['volumes'].T],['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df_rand, auto_alpha_rand = autoAlphaChooser([alphas_S_1to10_rand[v_i]['volumes'].T,alphas_D_1to10_rand[v_i]['volumes'].T],['Tegu random 1/'+denom,'Opossum random 1/'+denom],18, 10.0, 'convex', 'decreasing')
    volumes_df.loc[:,['Tegu uniform 1/'+denom,'Opossum uniform 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'])
    volumes_df_rand.loc[:,['Tegu random 1/'+denom,'Opossum random 1/'+denom]].plot(ax=ax[a_i], color=['C0','C1'], linestyle='dotted')
    volumes_df.loc[:,[volumes_df.columns[-1]]].plot(secondary_y=True, style='g',ax=ax[a_i])
    volumes_df_rand.loc[:,[volumes_df_rand.columns[-1]]].plot(secondary_y=True, style='#0D2913',ax=ax[a_i])
    x_bounds = ax[a_i].get_xlim()
    ax[a_i].legend(fontsize=10)
    ax[a_i].title.set_text('1 in '+denom)
    yMax.append(volumes_df.max().max())

maxMax = max(yMax)

ax[0,0].set_ylabel('Mobility volume (°^3)')
ax[1,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_ylabel('Mobility volume (°^3)')
ax[2,0].set_xlabel('Alpha')
ax[2,1].set_xlabel('Alpha')
ax[0,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[1,1].right_ax.set_ylabel('Tegu-opossum ratio')
ax[2,1].right_ax.set_ylabel('Tegu-opossum ratio')


ax[0,1].vlines(22, 0, maxMax, color='r')
ax[0,1].annotate(text='U,R 22', xy =(((22-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='center',size=8)

ax[1,0].vlines(24, 0, maxMax, color='r')
ax[1,0].vlines(26, 0, maxMax, color='r', linestyle='dotted')
ax[1,0].annotate(text='U 24', xy =(((24-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,0].annotate(text='R 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[1,1].vlines(26, 0, maxMax, color='r')
ax[1,1].vlines(28, 0, maxMax, color='r', linestyle='dotted')
ax[1,1].annotate(text='U 26', xy =(((26-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[1,1].annotate(text='R 28', xy =(((28-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,0].vlines(27, 0, maxMax, color='r')
ax[2,0].vlines(30, 0, maxMax, color='r', linestyle='dotted')
ax[2,0].annotate(text='U 27', xy =(((27-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[2,0].annotate(text='R 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)

ax[2,1].vlines(30, 0, maxMax, color='r')
ax[2,1].vlines(37, 0, maxMax, color='r', linestyle='dotted')
ax[2,1].annotate(text='U 30', xy =(((30-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='right',size=8)
ax[2,1].annotate(text='R 37', xy =(((37-x_bounds[0])/(x_bounds[1]-x_bounds[0])),1.01), xycoords='axes fraction', c = 'red', horizontalalignment='left',size=8)



## automate choosing alpha

In [ ]:
volumes_df = alpha_d['volumes'].T.join(alpha_s['volumes'].T, lsuffix='opossum', rsuffix='tegu').replace(0, np.nan)
volumes_df.index = pd.to_numeric([index.replace('_','.') for index in volumes_df.index])
volumes_df.columns=['opossum','tegu']
volumes_df['t-o ratio'] = volumes_df['tegu']/volumes_df['opossum']
volumes_df['ratio d1'] = np.gradient(volumes_df['t-o ratio'])
zero_crossings = [volumes_df.index[index] for index in np.where(np.diff(np.sign(volumes_df['ratio d1'])))[0]]
zero_crossing_rows = volumes_df.loc[zero_crossings].loc[(volumes_df['tegu'] >= volumes_df['tegu'].max()/2)&(volumes_df['opossum'] >= volumes_df['opossum'].max()/2)]
first_zero_crossing_row = zero_crossing_rows.iloc[0]
second_zero_crossing_row = zero_crossing_rows.iloc[1]



fig, ax = plt.subplots()
volumes_df.loc[:,['opossum','tegu']].plot(ax=ax, style=['blue','orange'])
volumes_df.loc[:,['t-o ratio']].plot(secondary_y=True, style='g',ax=ax, label='tegu-opossum ratio')
# volumes_df.loc[:,['ratio d1']].plot(secondary_y=True, style='g',ax=ax)
ax.vlines(second_zero_crossing_row.name, volumes_df.min().min(), volumes_df.max().max(), color='r', label='second 0 crossing of ratio d1')
ax.vlines(allEnclosed_d, volumes_df.min().min(), volumes_df.max().max(), color='blue', linestyle=':', label='opossum critical')
ax.vlines(allEnclosed_s, volumes_df.min().min(), volumes_df.max().max(), color='orange', linestyle=':', label='tegu critical')
ax.vlines(waterTight_d, volumes_df.min().min(), volumes_df.max().max(), color='blue', linestyle='dashed', label='opossum watertight')
ax.vlines(waterTight_s, volumes_df.min().min(), volumes_df.max().max(), color='orange', linestyle='dashed', label='tegu watertight')
ax.legend()

# SWAP CRX AND CRZ FOR ALL BELOW

In [ ]:
#center and superimpose joint alpha shapes
def centerJoints(df, species):
    df = df[df['species']==species[0]].copy()
    RCols = [col for col in df.columns if re.match(r'(c*R[xyz])', col)]
    for col in RCols:
        df[col+'_centered'] = df[col]-df[col].mean()
    return df
    
salCentered = centerJoints(df, 's')
didCentered = centerJoints(df, 'd')

rotationsCentered = getRotationRanges(salCentered, didCentered)
CmaxRx, CmaxcRz, CminRx, CmincRz, CmaxRy, CminRy, CmaxRz, CminRz = rotationsCentered.values()

alpha = 25
alphaSalVert,alphaSalEdge, alphaSalTri = alpha_shape_3D(np.array([salCentered.cRz_centered,salCentered.Ry_centered,salCentered.Rz_centered]).T,alpha)
alphaDidVert,alphaDidEdge, alphaDidTri = alpha_shape_3D(np.array([didCentered.cRz_centered,didCentered.Ry_centered,didCentered.Rz_centered]).T,alpha)


plt.close('all')


fig = plt.figure(figsize=[15,10], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')



format3dPlot(ax0, 'Tegu', (CmincRz, CmaxcRz), (CminRy, CmaxRy), (CminRz, CmaxRz), view3Q)
format3dPlot(ax1, 'Opossum', (CmincRz, CmaxcRz), (CminRy, CmaxRy), (CminRz, CmaxRz), view3Q)

addCosGrid(ax0, (CminRx,CmaxRx), (CminRy,CmaxRy), (CminRz,CmaxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (CminRx,CmaxRx), (CminRy,CmaxRy), (CminRz,CmaxRz), 5, xLevels=1, color='grey', linewidths=0.1)


ax0.plot_trisurf(salCentered.cRz_centered, salCentered.Ry_centered, alphaSalTri, salCentered.Rz_centered, shade=True, color=teguColor, linewidth=0, antialiased=True)
ax1.plot_trisurf(didCentered.cRz_centered, didCentered.Ry_centered, alphaDidTri, didCentered.Rz_centered, shade=True, color=opossumColor, linewidth=0, antialiased=True)

ax0.scatter(salCentered.cRz_centered,salCentered.Ry_centered,salCentered.Rz_centered, s=2, c=orangeYellowSeafoam[6], depthshade=False, edgecolors='None', alpha=0.95)
ax1.scatter(didCentered.cRz_centered,didCentered.Ry_centered,didCentered.Rz_centered, s=2, c=orangeYellowSeafoam[2], depthshade=False, edgecolors='None', alpha=0.95)


# ax0.add_patch(PolygonPatch(hullDXZ, fill=True, color='blue'))






connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()
# plt.savefig('3q.png', dpi=150, transparent=True)

In [ ]:
scaled_Zma_cols = [col for col in scaled_t_ma_all_data.columns if col[-3:] == 'Zma']
colsForForr = scaled_Zma_cols
# dfSforCorr = dfS.filter(colsForForr)
# dfSforCorr = dfSforCorr.dropna(axis=1, how='all')
# corr = dfSforCorr.corr()
# corr.style.background_gradient(cmap='bwr')
dfDforCorr = dfD.filter(colsForForr)
dfDforCorr_colnames_raw = [col.split('_',1)[1].rsplit('_',1)[0] for col in colsForForr]
dfDforCorr.columns = dfDforCorr_colnames_raw
dfDforCorr = dfDforCorr.dropna(axis=1, how='all')
corr = dfDforCorr.corr()
corr.style.background_gradient(cmap='bwr')

In [ ]:
from scipy.spatial import Delaunay
import numpy as np
from collections import defaultdict

def alpha_shape_3D(pos, alpha):
    """
    Compute the alpha shape (concave hull) of a set of 3D points.
    Parameters:
        pos - np.array of shape (n,3) points.
        alpha - alpha value.
    return
        outer surface vertex indices, edge indices, and triangle indices
    """

    tetra = Delaunay(pos)
    # Find radius of the circumsphere.
    # By definition, radius of the sphere fitting inside the tetrahedral needs 
    # to be smaller than alpha value
    # http://mathworld.wolfram.com/Circumsphere.html
    tetrapos = np.take(pos,tetra.vertices,axis=0)
    normsq = np.sum(tetrapos**2,axis=2)[:,:,None]
    ones = np.ones((tetrapos.shape[0],tetrapos.shape[1],1))
    a = np.linalg.det(np.concatenate((tetrapos,ones),axis=2))
    Dx = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[1,2]],ones),axis=2))
    Dy = -np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,2]],ones),axis=2))
    Dz = np.linalg.det(np.concatenate((normsq,tetrapos[:,:,[0,1]],ones),axis=2))
    c = np.linalg.det(np.concatenate((normsq,tetrapos),axis=2))
    r = np.sqrt(Dx**2+Dy**2+Dz**2-4*a*c)/(2*np.abs(a))
    # Find tetrahedrals
    tetras = tetra.vertices[r<alpha,:]
    # triangles
    TriComb = np.array([(0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3)])
    Triangles = tetras[:,TriComb].reshape(-1,3)
    Triangles = np.sort(Triangles,axis=1)

    # Remove triangles that occurs twice, because they are within shapes
    TrianglesDict = defaultdict(int)
    for tri in Triangles:TrianglesDict[tuple(tri)] += 1
    Triangles=np.array([tri for tri in TrianglesDict if TrianglesDict[tri] ==1])
    #edges
    EdgeComb=np.array([(0, 1), (0, 2), (1, 2)])
    Edges=Triangles[:,EdgeComb].reshape(-1,2)
    Edges=np.sort(Edges,axis=1)
    Edges=np.unique(Edges,axis=0)

    Vertices = np.unique(Edges)
    return Vertices,Edges,Triangles

In [ ]:
dfD[(dfD['Rx']>0)&(dfD['Rx']<5)&(dfD['Ry']>45)&(dfD['Ry']<50)][['scaled_dac_Xma','scaled_dac_Yma','scaled_dac_Zma']]

In [ ]:
#color by momentarm

def RgbMa(df, species, muscleName, xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.filter(like=muscleName)
    df = df.replace(np.NaN, 0)
    Xma = df.filter(like='Xma')
    Yma = df.filter(like='Yma')
    Zma = df.filter(like='Zma')
    df['posXma'] = np.where(Xma > 0, Xma, 0)
    df['posYma'] = np.where(Yma > 0, Yma, 0)
    df['posZma'] = np.where(Zma > 0, Zma, 0)
    df['negXma'] = np.where(Xma < 0, abs(Xma), 0)
    df['negYma'] = np.where(Yma < 0, abs(Yma), 0)
    df['negZma'] = np.where(Zma < 0, abs(Zma), 0)
    df=df.drop([Xma.columns[0],Yma.columns[0],Zma.columns[0]], axis=1)
    mmMax = df.max().max()
    df=df/mmMax
    df['Xma'] = df['posXma'] if xPos else df['negXma']
    df['Yma'] = df['posYma'] if yPos else df['negYma']
    df['Zma'] = df['posZma'] if zPos else df['negZma']
    totalXYZ = df[['Xma','Yma','Zma']].sum(axis=1)
    df['scale'] = preprocessing.MinMaxScaler().fit_transform(totalXYZ.values.reshape(-1, 1))
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species[0]]
    return(df[['Xma','Yma','Zma','scale']])



# stanceS = RgbMa(df, 'salvator', 'scaled_dac', xPos=True, yPos=False, zPos=False, normalizeAcrossSpecies=True)
# stanceD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=True, yPos=False, zPos=False, normalizeAcrossSpecies=True)
# swingS = RgbMa(df, 'salvator', 'scaled_dac', xPos=False, yPos=True, zPos=True, normalizeAcrossSpecies=True)
# swingD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=False, yPos=True, zPos=True, normalizeAcrossSpecies=True)

stanceS = RgbMa(df, 'salvator', 'scaled_dac', xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True)
stanceD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=True, yPos=True, zPos=True, normalizeAcrossSpecies=True)
swingS = RgbMa(df, 'salvator', 'scaled_dac', xPos=False, yPos=False, zPos=False, normalizeAcrossSpecies=True)
swingD = RgbMa(df, 'didelphis', 'scaled_dac', xPos=False, yPos=False, zPos=False, normalizeAcrossSpecies=True)

plt.close('all')


fig = plt.figure(figsize=[12,12], constrained_layout=True)
ax0 = fig.add_subplot(221, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(222, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(223, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(224, projection='3d', proj_type = 'ortho')


# 'Tegu Stance R: Prn, G: Add, B: Ext'
# 'Opossum Stance R: Prn, G: Add, B: Ext'

format3dPlot(ax0, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)
format3dPlot(ax1, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)
format3dPlot(ax2, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)
format3dPlot(ax3, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXZ)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=stanceS['scale']*15, c=stanceS[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=stanceD['scale']*15, c=stanceD[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=swingS['scale']*15, c=swingS[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=swingD['scale']*15, c=swingD[['Xma','Yma','Zma']], vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection4 = fig.canvas.mpl_connect('motion_notify_event', on_move4)

plt.show()

#threshold, then pca?

# plt.savefig('dsc.png', dpi=150, transparent=True)


In [ ]:

def divergingMa(df, species, muscleName, normalizeAcrossSpecies=True):
    df = df.copy()
    dfspecies = df.filter(items=['species'])
    if not normalizeAcrossSpecies: 
        df = df[df['species']==species[0]]
    df = df.filter(like=muscleName)
    df = df.replace(np.NaN, 0)
    mmMax = df.max().max()
    mmAbsMax = abs(df).max().max()
    mmMin = df.min().min()
    mmAbsMin = abs(df).min().min()
    MaxMinMax = max((abs(mmMax),abs(mmMin)))
    def MinMaxScalerPrescribedRange(X, min_input, max_input, min_output=0, max_output=1):
        X_std = (X - min_input) / (max_input - min_input)
        X_scaled = X_std * (max_output - min_output) + min_output
        return X_scaled
    Xma = df.filter(like='Xma')/MaxMinMax   #maxabs scale across all 3 axes
    Yma = df.filter(like='Yma')/MaxMinMax
    Zma = df.filter(like='Zma')/MaxMinMax
    df['Xmag'] = abs(Xma)   #magnitude of moment arm
    df['Ymag'] = abs(Yma)
    df['Zmag'] = abs(Zma)
    df['Xscale'] = MinMaxScalerPrescribedRange(Xma, -1, 1)    #get color mapping var by normalizing moment arms to lie between 0 and 1 (with -1 and 1 as min and max input values)
    df['Yscale'] = MinMaxScalerPrescribedRange(Yma, -1, 1)
    df['Zscale'] = MinMaxScalerPrescribedRange(Zma, -1, 1)
    if normalizeAcrossSpecies:
        df = dfspecies.join(df)
        df = df[df['species']==species[0]]
    return(df)



pec_crn_S = divergingMa(df, 'salvator', 'scaled_dac', normalizeAcrossSpecies=True)
pec_crn_D = divergingMa(df, 'didelphis', 'scaled_dac', normalizeAcrossSpecies=True)

plt.close('all')


fig = plt.figure(figsize=[12,18], constrained_layout=True)
ax0 = fig.add_subplot(321, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(322, projection='3d', proj_type = 'ortho')
ax2 = fig.add_subplot(323, projection='3d', proj_type = 'ortho')
ax3 = fig.add_subplot(324, projection='3d', proj_type = 'ortho')
ax4 = fig.add_subplot(325, projection='3d', proj_type = 'ortho')
ax5 = fig.add_subplot(326, projection='3d', proj_type = 'ortho')


# 'Tegu Stance R: Prn, G: Add, B: Ext'
# 'Opossum Stance R: Prn, G: Add, B: Ext'

format3dPlot(ax0, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax1, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax2, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax3, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax4, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)
format3dPlot(ax5, '', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), view3Q)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax2, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax3, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax4, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax5, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Xmag']*30, c=pec_crn_S['Xscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Xmag']*30, c=pec_crn_D['Xscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax2.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Ymag']*30, c=pec_crn_S['Yscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax3.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Ymag']*30, c=pec_crn_D['Yscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax4.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=pec_crn_S['Zmag']*30, c=pec_crn_S['Zscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')
ax5.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=pec_crn_D['Zmag']*30, c=pec_crn_D['Zscale'], cmap='seismic', vmin=0, vmax=1, depthshade=False, edgecolors='None')

connection6 = fig.canvas.mpl_connect('motion_notify_event', on_move6)

plt.show()

# plt.savefig('dac_div_3Q.png', dpi=150, transparent=True)




In [ ]:
#color by momentarm
dfRgb = df.copy()
dfSrgb = df[df['species']=='s']
dfDrgb = df[df['species']=='d']

scaled_muscle_cols = [col for col in dfRgb.columns if col.split('_')[0] == 'scaled' and col[-2:] == 'ma']
dfRgb[scaled_muscle_cols] = dfRgb[scaled_muscle_cols].replace(np.NaN, 0)

lat_crn = dfRgb[scaled_muscle_cols].iloc[:, 3:6]
latmax = lat_crn.max().max()
latmin = lat_crn.min().min()

latNewX = lat_crn.scaled_lat_crn_Xma+abs(latmin)
latNewY = lat_crn.scaled_lat_crn_Yma+abs(latmin)
latNewZ = lat_crn.scaled_lat_crn_Zma+abs(latmin)

latNewXnorm = latNewX/(latmax+abs(latmin))
latNewYnorm = latNewY/(latmax+abs(latmin))
latNewZnorm = latNewZ/(latmax+abs(latmin))



# scaler = sklearn.preprocessing.MinMaxScaler().fit(dfRgb[scaled_muscle_cols])
# dfRgb[scaled_muscle_cols] = scaler.transform(dfRgb[scaled_muscle_cols])
# DrgbXYZ = scaler.transform(dfDrgb[['scaled_Tx','scaled_Ty','scaled_Tz']])

# # SrgbXYZ = np.array([SrgbX, SrgbY, SrgbZ]).reshape(3,len(SrgbX)).T
# # DrgbXYZ = np.array([DrgbX, DrgbY, DrgbZ]).reshape(3,len(DrgbX)).T
plt.close('all')


fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)
format3dPlot(ax1, 'Opossum', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=2, c='black', depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=2, c='black', depthshade=False, edgecolors='None')
ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=2, c=np.array([np.zeros(len(latNewXnorm)),latNewYnorm,np.zeros(len(latNewXnorm)),latNewXnorm]).T[dfSrgb.index], depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=2, c=np.array([np.zeros(len(latNewXnorm)),latNewYnorm,np.zeros(len(latNewXnorm)),latNewXnorm]).T[dfDrgb.index], depthshade=False, edgecolors='None')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()

In [ ]:
 dfRgb[scaled_muscle_cols]

In [ ]:
#color by translation
dfRgb = df.copy()
# dfSrgb = df[df['species']=='s']
# dfDrgb = df[df['species']=='d']

scaled_Ts = dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']]
tMin = scaled_Ts.min().min()
tMax = scaled_Ts.max().max()

scalerMinMax = preprocessing.MinMaxScaler((tMin,tMax)).fit(dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']])
dfRgbMinMax = scalerMinMax.transform(dfRgb[['scaled_Tx','scaled_Ty','scaled_Tz']])
scaler01 = preprocessing.MinMaxScaler((0,1)).fit(dfRgbMinMax)
dfRgb01 = scaler01.transform(dfRgbMinMax)

dfSrgb = dfRgb01[dfRgb[dfRgb['species']=='s'].index]
dfDrgb = dfRgb01[dfRgb[dfRgb['species']=='d'].index]


plt.close('all')


fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0 = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1 = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0, 'Tegu', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)
format3dPlot(ax1, 'Opossum', (mincRx, maxcRx), (minRy, maxRy), (minRz, maxRz), viewXY)

addCosGrid(ax0, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)
addCosGrid(ax1, (minRx,maxRx), (minRy,maxRy), (minRz,maxRz), 5, xLevels=1, color='grey', linewidths=0.1)

ax0.scatter(dfS.cRx,dfS.Ry,dfS.Rz, s=3, c=dfSrgb, depthshade=False, edgecolors='None')
ax1.scatter(dfD.cRx,dfD.Ry,dfD.Rz, s=3, c=dfDrgb, depthshade=False, edgecolors='None')

connection1 = fig.canvas.mpl_connect('motion_notify_event', on_move)

plt.show()


In [ ]:
sklearn.preprocessing

In [ ]:
#PCA
from sklearn.decomposition import PCA

XYZ = df[['cRx','Ry','Rz']].to_numpy()
XYZ = sklearn.preprocessing.normalize(XYZ)
aPCA = PCA().fit(XYZ)

sXYZ = dfS[['cRx','Ry','Rz']].to_numpy()
sXYZ = sklearn.preprocessing.normalize(sXYZ)
sPCA = PCA().fit(sXYZ)

dXYZ = dfD[['cRx','Ry','Rz']].to_numpy()
dXYZ = sklearn.preprocessing.normalize(dXYZ)
dPCA = PCA().fit(dXYZ)


aPCAs = aPCA.transform(sXYZ).T
aPCAd = aPCA.transform(dXYZ).T
sPCAs = sPCA.transform(sXYZ).T
sPCAd = sPCA.transform(dXYZ).T
dPCAs = dPCA.transform(sXYZ).T
dPCAd = dPCA.transform(dXYZ).T

maxPC1a = max([max(aPCAs[0]),max(aPCAd[0])])
minPC1a = min([min(aPCAs[0]),min(aPCAd[0])])
maxPC2a = max([max(aPCAs[1]),max(aPCAd[1])])
minPC2a = min([min(aPCAs[1]),min(aPCAd[1])])
maxPC3a = max([max(aPCAs[2]),max(aPCAd[2])])
minPC3a = min([min(aPCAs[2]),min(aPCAd[2])])

maxPC1s = max([max(sPCAs[0]),max(sPCAd[0])])
minPC1s = min([min(sPCAs[0]),min(sPCAd[0])])
maxPC2s = max([max(sPCAs[1]),max(sPCAd[1])])
minPC2s = min([min(sPCAs[1]),min(sPCAd[1])])
maxPC3s = max([max(sPCAs[2]),max(sPCAd[2])])
minPC3s = min([min(sPCAs[2]),min(sPCAd[2])])

maxPC1d = max([max(dPCAs[0]),max(dPCAd[0])])
minPC1d = min([min(dPCAs[0]),min(dPCAd[0])])
maxPC2d = max([max(dPCAs[1]),max(dPCAd[1])])
minPC2d = min([min(dPCAs[1]),min(dPCAd[1])])
maxPC3d = max([max(dPCAs[2]),max(dPCAd[2])])
minPC3d = min([min(dPCAs[2]),min(dPCAd[2])])

plt.close('all')



fig = plt.figure(figsize=[12,5], constrained_layout=True)
ax0PCA = fig.add_subplot(121, projection='3d', proj_type = 'ortho')
ax1PCA = fig.add_subplot(122, projection='3d', proj_type = 'ortho')

format3dPlot(ax0PCA, 'Tegu', (minPC1a, maxPC1a), (minPC2a, maxPC2a), (minPC3a, maxPC3a), viewXY, mode='pca')
format3dPlot(ax1PCA, 'Opossum', (minPC1a, maxPC1a), (minPC2a, maxPC2a), (minPC3a, maxPC3a), viewXY, mode='pca')

ax0PCA.scatter(aPCAs[0],aPCAs[1],aPCAs[2], s=3, depthshade=True)
ax1PCA.scatter(aPCAd[0],aPCAd[1],aPCAd[2], s=3, depthshade=True)



plt.show()


In [ ]:
def alpha_shape(points, alpha, only_outer=True): #c/o "Harold" on stackoverflow
    """
    Compute the alpha shape (concave hull) of a set of points.
    :param points: np.array of shape (n, 3) points.
    :param alpha: alpha value.
    :param only_outer: boolean value to specify if we keep only the outer border
    or also inner edges.
    :return: set of (i,j) pairs representing edges of the alpha-shape. (i,j) are
    the indices in the points array.
    """
    assert points.shape[0] > 3, "Need at least four points"

    def add_edge(edges, i, j):
        """
        Add a line between the i-th and j-th points,
        if not in the set already
        """
        if (i, j) in edges or (j, i) in edges:
            # already added
            if only_outer:
                # if both neighboring triangles are in shape, it's not a boundary edge
                if (j, i) in edges:
                    edges.remove((j, i))
            return
        edges.add((i, j))

    tri = Delaunay(points)
    edges = set()
    # Loop over triangles:
    # ia, ib, ic, id = indices of corner points of the tetrahedron
    print(tri.vertices.shape)
    for ia, ib, ic, id in tri.vertices:
        pa = points[ia]
        pb = points[ib]
        pc = points[ic]
        pd = points[id]

        # Computing radius of tetrahedron Circumsphere
        # http://mathworld.wolfram.com/Circumsphere.html

        pa2 = np.dot(pa, pa)
        pb2 = np.dot(pb, pb)
        pc2 = np.dot(pc, pc)
        pd2 = np.dot(pd, pd)

        a = np.linalg.det(np.array([np.append(pa, 1), np.append(pb, 1), np.append(pc, 1), np.append(pd, 1)]))

        Dx = np.linalg.det(np.array([np.array([pa2, pa[1], pa[2], 1]),
                                     np.array([pb2, pb[1], pb[2], 1]),
                                     np.array([pc2, pc[1], pc[2], 1]),
                                     np.array([pd2, pd[1], pd[2], 1])]))

        Dy = - np.linalg.det(np.array([np.array([pa2, pa[0], pa[2], 1]),
                                       np.array([pb2, pb[0], pb[2], 1]),
                                       np.array([pc2, pc[0], pc[2], 1]),
                                       np.array([pd2, pd[0], pd[2], 1])]))

        Dz = np.linalg.det(np.array([np.array([pa2, pa[0], pa[1], 1]),
                                     np.array([pb2, pb[0], pb[1], 1]),
                                     np.array([pc2, pc[0], pc[1], 1]),
                                     np.array([pd2, pd[0], pd[1], 1])]))

        c = np.linalg.det(np.array([np.array([pa2, pa[0], pa[1], pa[2]]),
                                    np.array([pb2, pb[0], pb[1], pb[2]]),
                                    np.array([pc2, pc[0], pc[1], pc[2]]),
                                    np.array([pd2, pd[0], pd[1], pd[2]])]))

        circum_r = math.sqrt(math.pow(Dx, 2) + math.pow(Dy, 2) + math.pow(Dz, 2) - 4 * a * c) / (2 * abs(a))
        if circum_r < alpha:
            add_edge(edges, ia, ib)
            add_edge(edges, ib, ic)
            add_edge(edges, ic, id)
            add_edge(edges, id, ia)
            add_edge(edges, ia, ic)
            add_edge(edges, ib, id)
    return edges

## Deprecated
### other tests for alpha quality
#### check if all vertices are enclosed, returns same index as criticality test

In [ ]:
def getVerticesForAlpha(vertices_df, alpha):
    alpha_vertices_df = vertices_df.copy().loc[vertices_df[alpha]<3]
    alpha_vertices_df['subindex'] = range(len(alpha_vertices_df))
    return alpha_vertices_df
def getTrianglesForAlpha(triangles_df, alpha):
    alpha_triangles_df = triangles_df.copy().loc[triangles_df[alpha]==2.0]
    alpha_triangles_df['subindex'] = range(len(alpha_triangles_df))
    return alpha_triangles_df
def getNeighborTriangles(alpha_triangles_df, triangle_index):
    alpha_triangle_vertices = list(alpha_triangles_df.loc[triangle_index][['index1','index2','index3']])
    other_alpha_triangles_df = alpha_triangles_df.drop(triangle_index)
    neighbor_triangles_df = other_alpha_triangles_df.loc[(other_alpha_triangles_df['index1'].isin(alpha_triangle_vertices))|(other_alpha_triangles_df['index2'].isin(alpha_triangle_vertices)|(other_alpha_triangles_df['index3'].isin(alpha_triangle_vertices)))]
    return neighbor_triangles_df

# allEnclosed_d = float([key for key in params_d if params_d[key]['allEnclosed']][0].replace('_','.'))
# allEnclosed_s = float([key for key in params_s if params_s[key]['allEnclosed']][0].replace('_','.'))
# waterTight_d = float([key for key in params_d if 'waterTight' in params_d[key].keys() and params_d[key]['waterTight']][0].replace('_','.'))
# waterTight_s = float([key for key in params_s if 'waterTight' in params_s[key].keys() and params_s[key]['waterTight']][0].replace('_','.'))

In [ ]:
alpha_s_model


#### check watertightness

In [ ]:
# %pip install trimesh
import trimesh
# trimesh.util.attach_to_log()

def getVertexFaceRepresentationForAlpha(alphaObject_df, alpha):
    verts = getVerticesForAlpha(alphaObject_df['vertices'],alpha)[['x','y','z','index','subindex']]
    triangles = getTrianglesForAlpha(alphaObject_df['triangles'],alpha)[['index1','index2','index3','subindex']]
    for i in triangles.index:
        for indexn in ['index1','index2','index3']:
            index_to_find = float(triangles.loc[i, indexn])
            triangles.loc[i, indexn] = int(verts.loc[verts['index']==index_to_find]['subindex'])
    return([np.array(verts[['x','y','z']]), np.array(triangles[['index1','index2','index3']], dtype='int')])
def paramsByAlpha(alphaObject_df, alphaCols):
    result = {}
    for alpha in alphaCols:
#         print('testing '+alpha)
        encloses_all_vertices = np.all(alphaObject_df['vertices'][alpha]<3.0)
        volume = alphaObject_df['volumes'][alpha]
        if encloses_all_vertices:
            verts, triangles = getVertexFaceRepresentationForAlpha(alphaObject_df, alpha)
            mesh = trimesh.Trimesh(vertices=verts, faces=triangles, process=True)
            waterTight = mesh.is_watertight
            euler_characteristic = mesh.euler_number
            result[alpha] = {'allEnclosed':encloses_all_vertices, 'volume': volume, 'waterTight':waterTight,'euler':euler_characteristic}
        else:
            result[alpha] = {'allEnclosed':encloses_all_vertices, 'volume': volume}
    return result

# params_d = paramsByAlpha(alpha_d, alphaCols)
# params_s = paramsByAlpha(alpha_s, alphaCols)

### JCS conversion

In [ ]:
# plt.close('all')
# dfAll3 = pd.read_csv('/Users/phil/Downloads/sundaycheckboth.csv', index_col=0)
# dfAll3_p= dfAll3.filter(regex='philJCSdata.r')
# dfAll3_rD = dfAll3.filter(regex='rob_ddata.r')
# dfAll3_rS = dfAll3.filter(regex='rob_sdata.r')

# dfAll3_p.columns=['Rx','Ry','Rz']
# dfAll3_rD.columns=['Rx','Ry','Rz']
# dfAll3_rS.columns=['Rx','Ry','Rz']


# phil_mat = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]
# rob_tegu_mat = np.array([[0.922104644399, -0.191552602191, -0.336200275687, 0.0], [0.0969171034844, 0.955507396629, -0.278590541905, 0.0], [0.374606593416, 0.224306075666, 0.899642531558, 0.0], [7.24897435154e-17, 1.6897613437e-16, -2.22044604925e-16, 1.0]])[0:3,0:3]
# rob_opossum_mat = np.array([[0.98966482419, -0.0841148709321, 0.116138814565, 0.0], [-0.0345598571996, 0.646116773228, 0.762455724369, 0.0], [-0.13917310096, -0.758589351258, 0.636532044855, 0.0], [7.83488092761e-17, 1.03588051488e-16, -6.66133814775e-16, 1.0]])[0:3,0:3]

# tegu_converted = convertDataBetweenJCS(phil_mat, rob_tegu_mat, dfAll3_rS)
# opossum_converted = convertDataBetweenJCS(phil_mat, rob_opossum_mat, dfAll3_rD)


# fig, ax = plt.subplots(1,3, figsize=(15,5))
# dfAll3_p.plot(style=['r','g','b'], ax=ax[0],title='Target (Phil)')
# tegu_converted.plot(style=['r','g','b'], ax=ax[1],title='Rob Tegu')
# opossum_converted.plot(style=['r','g','b'], ax=ax[2],title='Rob Opossum')

In [ ]:
# ## actual tegu matrix
# df_teguComp = pd.read_csv('/Users/phil/Desktop/rotoscoping/Utility scenes/tegutest.csv', index_col=0)
# df_teguComp_p= df_teguComp.filter(regex='philJCSdata.r')
# df_teguComp_rS = df_teguComp.filter(regex='robJCSdata.r')

# df_teguComp_p.columns=['Rx','Ry','Rz']
# df_teguComp_rS.columns=['Rx','Ry','Rz']


# phil_mat = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]

# tegu_converted = convertDataBetweenJCS(phil_mat, rob_tegu_matrix, df_teguComp_rS)


# fig, ax = plt.subplots(1,2, figsize=(15,5))
# df_teguComp_p.plot(style=['r','g','b'], ax=ax[0],title='Target (Phil)')
# tegu_converted.plot(style=['r','g','b'], ax=ax[1],title='Rob Tegu')


In [ ]:
# def convertDataBetweenJCSreversedCS(destination_matrix, source_matrix, source_df):
#     source_from_dest_matrix = np.matmul(source_matrix, destination_matrix.T)
#     rotations = Rotation.from_euler('XYZ', source_df.loc[:,['Rx','Ry','Rz']].to_numpy(), degrees=True)
#     converted = rotations*Rotation.from_matrix(source_from_dest_matrix)
#     eulers = converted.as_euler('XYZ', degrees=True)  
#     result_df = source_df.copy()
#     result_df.loc[:,['Rx','Ry','Rz']] = eulers
#     return result_df


# ## actual opossum matrix
# df_opossumComp = pd.read_csv('/Users/phil/Desktop/rotoscoping/Utility scenes/opossumtest.csv', index_col=0)
# df_opossumComp_p= df_opossumComp.filter(regex='philJCSdata.r')
# df_opossumComp_rD = df_opossumComp.filter(regex='newRobJaxdata.r')

# rob_opossum_matrix = np.array([[-0.988997647945, -0.0952296421902, -0.113203213768, 0.0], [0.144901479974, -0.777702952339, -0.611703914508, 0.0], [-0.0297861286566, -0.6213770459, 0.782945306754, 0.0], [0.0, -2.22044604925e-16, -8.881784197e-16, 1.0]])[0:3,0:3]


# df_opossumComp_p.columns=['Rx','Ry','Rz']
# df_opossumComp_rD.columns=['Rx','Ry','Rz']


# phil_mat = np.array([[1.0, 0.0, 0.0, 0.0], [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 1.0, 0.0], [0.0, 0.0, 0.0, 1.0]])[0:3,0:3]

# opossum_converted = convertDataBetweenJCSreversedCS(phil_mat, rob_opossum_matrix, df_opossumComp_rD)


# fig, ax = plt.subplots(1,2, figsize=(15,5))
# df_opossumComp_p.plot(style=['r','g','b'], ax=ax[0],title='Target (Phil)')
# opossum_converted.plot(style=['r','g','b'], ax=ax[1],title='Rob Opossum')
